In [ ]:
!pip install spacy==2.3.4
!python -m spacy download pt_core_news_lg

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd
import numpy as np
import random
import time
import pickle
import re
import spacy
from collections import Counter
import json
from random import randint
from spacy.matcher import Matcher
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm, trange
import nltk
import string
from datetime import datetime, timezone
from lxml.html.clean import Cleaner
cleaner = Cleaner(page_structure=True)

!pip install Unidecode
from unidecode import unidecode

nltk.download('punkt')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

import pymongo
from pymongo import MongoClient

from pathlib import Path
diretorio = Path('/content/gdrive/My Drive/NER/DataFrames/URLS')

     |████████████████████████████████| 235 kB 17.4 MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# DataFrames com URLS

In [ ]:
df_sentences = pd.DataFrame()

In [ ]:
df_sentences['preço NER'] = 'NENHUM'
df_sentences['dorms NER'] = 'NENHUM'
df_sentences['quartos NER'] = 'NENHUM'
df_sentences['vagas NER'] = 'NENHUM'
df_sentences['area NER'] = 'NENHUM'
df_sentences['suites NER'] = 'NENHUM'
df_sentences['banheiros NER'] = 'NENHUM'

df_sentences['preco REGEX'] = 'NENHUM'
df_sentences['quartos REGEX'] = 'NENHUM'
df_sentences['dorms REGEX'] = 'NENHUM'
df_sentences['vagas REGEX'] = 'NENHUM'
df_sentences['area REGEX'] = 'NENHUM'
df_sentences['suites REGEX'] = 'NENHUM'
df_sentences['banheiros REGEX'] = 'NENHUM'

df_sentences['preco REGEX correct'] = 'NENHUM'
df_sentences['quartos REGEX correct'] = 'NENHUM'
df_sentences['dorms REGEX correct'] = 'NENHUM'
df_sentences['vagas REGEX correct'] = 'NENHUM'
df_sentences['area REGEX correct'] = 'NENHUM'
df_sentences['area priv REGEX correct'] = 'NENHUM'
df_sentences['suites REGEX correct'] = 'NENHUM'
df_sentences['banheiros REGEX correct'] = 'NENHUM'

df_sentences['only area REGEX'] = 'NENHUM'
df_sentences['only area priv REGEX'] = 'NENHUM'
df_sentences['only area terreno REGEX'] = 'NENHUM'
df_sentences['only dorms REGEX'] = 'NENHUM'
df_sentences['only quartos REGEX'] = 'NENHUM'
df_sentences['only suites REGEX'] = 'NENHUM'
df_sentences['only vagas REGEX'] = 'NENHUM'
df_sentences['only banheiros REGEX'] = 'NENHUM'
df_sentences['only garagem REGEX'] = 'NENHUM'
df_sentences['only carros REGEX'] = 'NENHUM'

df_sentences['only dorms REGEX 2'] = 'NENHUM'
df_sentences['only quartos REGEX 2'] = 'NENHUM'
df_sentences['only suites REGEX 2'] = 'NENHUM'
df_sentences['only vagas REGEX 2'] = 'NENHUM'
df_sentences['only banheiros REGEX 2'] = 'NENHUM'
df_sentences['only garagem REGEX 2'] = 'NENHUM'
df_sentences['only carros REGEX 2'] = 'NENHUM'

df_sentences['only dorms REGEX 3'] = 'NENHUM'
df_sentences['only quartos REGEX 3'] = 'NENHUM'
df_sentences['only suites REGEX 3'] = 'NENHUM'
df_sentences['only vagas REGEX 3'] = 'NENHUM'
df_sentences['only banheiros REGEX 3'] = 'NENHUM'
df_sentences['only garagem REGEX 3'] = 'NENHUM'
df_sentences['only carros REGEX 3'] = 'NENHUM'

df_sentences['dorms REGEX sentence'] = 'NENHUM'
df_sentences['quartos REGEX sentence'] = 'NENHUM'
df_sentences['suites REGEX sentence'] = 'NENHUM'
df_sentences['vagas REGEX sentence'] = 'NENHUM'
df_sentences['carro REGEX sentence'] = 'NENHUM'
df_sentences['garagem REGEX sentence'] = 'NENHUM'
df_sentences['banheiros REGEX sentence'] = 'NENHUM'

df_sentences['dorms unique REGEX'] = 'NENHUM'
df_sentences['quartos unique REGEX'] = 'NENHUM'
df_sentences['suites unique REGEX'] = 'NENHUM'
df_sentences['vagas unique REGEX'] = 'NENHUM'
df_sentences['banheiros unique REGEX'] = 'NENHUM'
df_sentences['garagem unique REGEX'] = 'NENHUM'
df_sentences['carros unique REGEX'] = 'NENHUM'
#df_sentences['box unique REGEX'] = 'NENHUM'

df_sentences['Dorms Show'] = 'NENHUM'
df_sentences['Quartos Show'] = 'NENHUM'
df_sentences['Suites Show'] = 'NENHUM'
df_sentences['Vagas Show'] = 'NENHUM'
df_sentences['Banheiros Show'] = 'NENHUM'
df_sentences['Garagem Show'] = 'NENHUM'
df_sentences['Carros Show'] = 'NENHUM'

df_sentences['Area Show'] = 'NENHUM'
df_sentences['Area Priv Show'] = 'NENHUM'
df_sentences['Area Terreno Show'] = 'NENHUM'
df_sentences['Area Global Show'] = 'NENHUM'
df_sentences['Area Útil Show'] = 'NENHUM'
df_sentences['Area Construída Show'] = 'NENHUM'
df_sentences['Area Total Show'] = 'NENHUM'

df_sentences['Dorms OU'] = 'NENHUM'
df_sentences['Quartos OU'] = 'NENHUM'
df_sentences['Suites OU'] = 'NENHUM'
df_sentences['Banheiros OU'] = 'NENHUM'
df_sentences['Vagas OU'] = 'NENHUM'
df_sentences['Garagem OU'] = 'NENHUM'
df_sentences['Carros OU'] = 'NENHUM'
df_sentences['Box OU'] = 'NENHUM'

df_sentences['Preço All'] ='NENHUM'

In [ ]:
nlp = spacy.load('pt_core_news_lg')

In [ ]:
nlp4 = spacy.load('/content/gdrive/My Drive/NER/spacy_NER4')
nlp5 = spacy.load('/content/gdrive/My Drive/NER/spacy_NER5')
nlp6 = spacy.load('/content/gdrive/My Drive/NER/spacy_NER6')
nlp7 = spacy.load('/content/gdrive/My Drive/NER/spacy_NER7')
nlp71 = spacy.load('/content/gdrive/My Drive/NER/spacy_NER7.1')

/usr/local/lib/python3.7/dist-packages/spacy/_ml.py:304: UserWarning: [W019] Changing vectors name from pt_core_news_lg.vectors to pt_core_news_lg.vectors_0, to avoid clash with previously loaded vectors. See Issue #3853.
  warnings.warn(Warnings.W019.format(old=old_name, new=new_name))


In [ ]:
nlp8 = spacy.load('/content/gdrive/My Drive/NER/spacy_NER8') #preço
nlp9 = spacy.load('/content/gdrive/My Drive/NER/spacy_NER9') #dorms e quartos
nlp10 = spacy.load('/content/gdrive/My Drive/NER/spacy_NER10') #vagas
nlp11 = spacy.load('/content/gdrive/My Drive/NER/spacy_NER11') #área
nlp12 = spacy.load('/content/gdrive/My Drive/NER/spacy_NER12') #suítes e banheiros

# Funções para executar

In [ ]:
def returnComodosJuntos(s):
  s = re.sub(',', '', re.sub('\s+', ' ', unidecode(s.lower().replace('(','').replace(')','').replace(':','').replace('\xa0',''))))
  list_correct = []
  for regex in ['\w+\s*\d{1,2}\s*', '\d{1,2}\s*\w+\s*']:
    rgx = regex
    for qnt_vezes in range(1,5):
      rgx += regex
      values = re.findall(rgx, s)
      for v in values:
        v = v.strip()
        cont_ok = 0
        list_tags_ok = []
        for tag in ['dorm', 'suit', 'vaga', 'banheiro', 'quarto', 'garage']: #palavras chaves que estou procurando
          #if tag in v: cont_ok +=1
          #verifica se sobra letras/palavras além das tags procuradas. Se retornar, então não é válido
          if tag in v:
            list_tags_ok.append(tag)
            cont_ok +=1
        sr = v
        for tags_ok in list_tags_ok:
          sr = re.sub(tags_ok+'\w*\D', '', sr)
        sr = re.sub('\s+', '', sr)
        if len(re.findall('\D', sr)) > 0:
          #print("OPAAAAAA, NÃO DÁ", v, "SR", sr)
          cont_ok = -1
        list_correct.append((cont_ok, v, regex))

  list_correct.sort(reverse=True)
  #print("correct", list_correct)
  dorms_rgx_only, suites_rgx_only, vagas_rgx_only, banheiro_rgx_only, quarto_rgx_only, garagem_rgx_only = 'NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM'
  frase_correct, regex_correct = '',''
  if len(list_correct) > 0:
    frase_correct = list_correct[0][1]
    regex_correct = list_correct[0][2]
    if regex_correct == '\\d{1,2}\\s*\\w+\\s*':
      if (re.findall('\d{1,2}\s*dorm', frase_correct)) != []:
        dorms_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*dorm', frase_correct)[0])[0]
      if (re.findall('\d{1,2}\s*suit', frase_correct)) != []:
        suites_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*suit', frase_correct)[0])[0]
      if (re.findall('\d{1,2}\s*vaga', frase_correct)) != []:
        vagas_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*vaga', frase_correct)[0])[0]
      if (re.findall('\d{1,2}\s*banheiro', frase_correct)) != []:
        banheiro_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*banheiro', frase_correct)[0])[0]
      if (re.findall('\d{1,2}\s*quarto', frase_correct)) != []:
        quarto_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*quarto', frase_correct)[0])[0]
      if (re.findall('\d{1,2}\s*garage', frase_correct)) != []:
        garagem_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*garage', frase_correct)[0])[0]
    elif regex_correct == '\\w+\\s*\\d{1,2}\\s*':
      if (re.findall('dorm\w*\s*\d{1,2}', frase_correct)) != []:
        dorms_rgx_only = re.findall('\d{1,2}', re.findall('dorm\w*\s*\d{1,2}', frase_correct)[0])[0]
      if (re.findall('suit\w*\s*\d{1,2}', frase_correct)) != []:
        suites_rgx_only = re.findall('\d{1,2}', re.findall('suit\w*\s*\d{1,2}', frase_correct)[0])[0]
      if (re.findall('vaga\w*\s*\d{1,2}', frase_correct)) != []:
        vagas_rgx_only = re.findall('\d{1,2}', re.findall('vaga\w*\s*\d{1,2}', frase_correct)[0])[0]
      if (re.findall('banheiro\w*\s*\d{1,2}', frase_correct)) != []:
        banheiro_rgx_only = re.findall('\d{1,2}', re.findall('banheiro\w*\s*\d{1,2}', frase_correct)[0])[0]
      if (re.findall('quarto\w*\s*\d{1,2}', frase_correct)) != []:
        quarto_rgx_only = re.findall('\d{1,2}', re.findall('quarto\w*\s*\d{1,2}', frase_correct)[0])[0]
      if (re.findall('garage\w*\s*\d{1,2}', frase_correct)) != []:
        garagem_rgx_only = re.findall('\d{1,2}', re.findall('garage\w*\s*\d{1,2}', frase_correct)[0])[0]
    
  return frase_correct, regex_correct, dorms_rgx_only, suites_rgx_only, vagas_rgx_only, banheiro_rgx_only, quarto_rgx_only, garagem_rgx_only

In [ ]:
def indicesArea(s):
  doc = nlp8(s)

  indices_area = []
  for regex_area in ['\d+.?,?\d+?\s*?m²', '\d+.?,?\d+?\s*?m2', '\d+.?,?\d+?\s*?m ²', '\d+.?,?\d+?\s*?m 2']:
    for match in re.finditer(regex_area, doc.text):
      start, end = match.span()
      span = doc.char_span(start, end, label='area')
      if span:
        indices_area += range(start, end+1)
  return indices_area

In [ ]:
def indicesPreco(s):
  doc = nlp8(s)

  indices_preco = []
  for match in re.finditer('\(?r\$\)?[:\s*]*\d+[.,]?[\d+]*?[.,]?[\d+]*[.,]?[\d+]*[.,]?[\d+]*', doc.text):
    start, end = match.span()
    span = doc.char_span(start, end, label='preco')
    if span:
      indices_preco += range(start, end+1)
  return indices_preco

In [ ]:
#AAAAAAAAQQQQQQQQQUIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII ------------------------------
def comodosUnique(s):
  s = re.sub('\s+', ' ', unidecode(s.lower().replace('(','').replace(')','').replace(':','').replace('\xa0','').replace('.', ' .')))
  doc = nlp8(s)
  for tag in ['dorm', 'suit', 'vaga', 'banheiro', 'quarto', 'garage', 'carro', 'box']:
    regex_tag = '\w*?'+tag+'\w*'
    list_values_rigth, list_values_left = [], []
    parte_direita_numero, parte_esquerda_numero = 'NENHUM', 'NENHUM'
    number_direita, number_esquerda = [], []
    indices_area = indicesArea(s)
    indices_preco = indicesPreco(s)
    value_dorms_unique = 'NENHUM'
    for match in re.finditer(regex_tag, doc.text):
      start, end = match.span()
      span = doc.char_span(start, end, label=tag)
      if span:
        #print(span)
        for k in range(1,10):
          if re.findall('\d+', s[start:end]) != []:                 #dorms2
            value_dorms_unique = re.findall('\d+', s[start:end])[0]
            break
          if end+k > len(s)-1:break
          frase = s[start:end+k]
          #print("frase", frase, frase[-1])
          if frase[-1] == '.' or frase[-1] == ',':
            break
          if frase[-1].isspace() == False:
            if frase[-1].isdigit() == True and s[start:end+k+1][-1].isdigit() == False:  #dorms 2
              if end+k in indices_preco or end+k in indices_area: break
              number_direita.append((re.findall('\d+', s[start:end+k])[0], [x for x in range(start, end+k+1)]))
              #print(s[start:end+k])
              parte_direita_numero = 'ok'
            elif frase[-1].isdigit() == True and s[start:end+k+1][-1].isdigit() == True: #dorms 02
              if end+k in indices_preco or end+k in indices_area: break
              number_direita.append((re.findall('\d+', s[start:end+k+1])[0], [x for x in range(start,end+k+2)]))
              #print(s[start:end+k+1])
              parte_direita_numero = 'ok'
            break
        for j in range(1,10):
          if start-j < 0:break
          if start-j-1 < 0:j +=1
          frase = s[start-j:end]
          if len(frase) == 0:break
          if frase[0] == '.' or frase[0] == ',':break
          if frase[0].isspace() == False:
            if start-j in indices_preco or start-j in indices_area: break
            if frase[0].isdigit() == True and s[start-j-1:end][0].isdigit() == False:  #2 dorms 
              number_esquerda.append( (re.findall('\d+', s[start-j:end])[0], [x for x in range(start-j, end+1)]))
              #print(s[start-j:end])
              parte_esquerda_numero = 'ok'
            if frase[0].isdigit() == True and s[start-j-1:end][0].isdigit() == True:  #02 dorms 
              if start-j in indices_preco or start-j in indices_area: break
              number_esquerda.append( (re.findall('\d+', s[start-j-1:end])[0], [x for x in range(start-j-1, end+1)]))
              #print(s[start-j-1:end])
              parte_esquerda_numero = 'ok'
            break
    
    number_unique_ok = 'NENHUM'

    dorms_unique, quartos_unique, suites_unique, vagas_unique, banheiros_unique, garagem_unique, carros_unique, box_unique = 'NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM'

    if value_dorms_unique != 'NENHUM' and parte_esquerda_numero != 'ok':
      number_unique_ok = value_dorms_unique

    if number_direita != [] and number_esquerda == []:
      #number_unique_ok = number_direita[0]
      for v in number_direita:
        v = v[0]
        if number_unique_ok == 'NENHUM':number_unique_ok = v
        else: number_unique_ok +='&-&'+v
    elif number_direita == [] and number_esquerda != []:
      #number_unique_ok = number_esquerda[0]
      for v in number_esquerda:
        v = v[0]
        if number_unique_ok == 'NENHUM':number_unique_ok = v
        else: number_unique_ok +='&-&'+v
    elif number_direita != [] and number_esquerda != []:
      #print(number_direita, number_esquerda)
      list_indices_inserido, list_final = [],[]
      for v1 in number_direita:
        dentro = 0
        for v2 in number_esquerda:
          if (v1[1][0] in v2[1] or v1[1][-1] in v2[1]):
            dentro = 1
            break
        if dentro == 0:
          list_indices_inserido.append(v1[1])
          list_final.append((v1[0], v1[1]))
          #print("ADD", v1[0], v1[1])
          for b1 in number_esquerda:
            dentro = 0
            #print(list_indices_inserido)
            for b2 in number_direita:
              if b1[1][0] in b2[1] or b1[1][-1] in b2[1] or b1[1] in list_indices_inserido:
                dentro = 1
                break
            if dentro == 0:
              #print("ADD", b1[0], b1[1])
              list_final.append((b1[0], b1[1]))
              break
          #print("ok", b1[1][0], b1[1][-1], b2[1], b1[0], b2[0])
        else:
          continue
      #print(list_final)
      list_final = [t[0] for t in list(set([( x[0], str(x[1])) for x in list_final]))]

      for v in list_final:
        if number_unique_ok == 'NENHUM':number_unique_ok = v
        else: number_unique_ok +='&-&'+v
      #print(list_final)

    if tag == 'dorm' and number_unique_ok != 'NENHUM':dorms_unique = number_unique_ok
    if tag == 'quarto' and number_unique_ok != 'NENHUM':quartos_unique = number_unique_ok
    if tag == 'banheiro' and number_unique_ok != 'NENHUM':banheiros_unique = number_unique_ok
    if tag == 'suit' and number_unique_ok != 'NENHUM':suites_unique = number_unique_ok
    if tag == 'vaga' and number_unique_ok != 'NENHUM':vagas_unique = number_unique_ok
    if tag == 'garage' and number_unique_ok != 'NENHUM':garagem_unique = number_unique_ok
    if tag == 'carro' and number_unique_ok != 'NENHUM':carros_unique = number_unique_ok
    if tag == 'box' and number_unique_ok != 'NENHUM':box_unique = number_unique_ok

    #print(tag, number_unique_ok)
    return dorms_unique, quartos_unique, suites_unique, vagas_unique, banheiros_unique, garagem_unique, carros_unique, box_unique

In [ ]:
list_tags_search = ['dorm', 'suit', 'vaga', 'banheiro', 'quarto', 'garage', 'carro', 'box']
def verify_tags_and_numbers(s, word):
  for tag in list_tags_search:
    if tag in word: 
      #print("word", word)
      return 1
  return 0

In [ ]:
list_tags_search = ['dorm', 'suit', 'vaga', 'banheiro', 'quarto', 'garage', 'carro', 'box']
def verify_tags_and_numbers(s, word):
  for tag in list_tags_search:
    if tag in word: 
      #print("word", word)
      return 1
  return 0

In [ ]:
def comodosChavesJuntos(s):
  indices_preco, indices_area = [], []
  s = re.sub('\s+', ' ', unidecode(s.lower().replace('(','').replace(')','').replace(':','').replace('\xa0','')))
  list_tags_search = ['dorm', 'suit', 'vaga', 'banheiro', 'quarto', 'garage', 'carro', 'box']
  list_return_frase = []
  doc = nlp8(s) 
  indices_preco = indicesPreco(s)
  indices_area = indicesArea(s)
  for tag in list_tags_search:
    regex_tag = tag+'\w*'
    number_direita, number_esquerda = [], []
    for match in re.finditer(regex_tag, doc.text):
      start, end = match.span()
      span = doc.char_span(start, end, label=tag)
      parte_direita_numero, parte_esquerda_numero = [], []
      if span:
        #print("SPAN", span)
        for k in range(1,50):                           #NÚMERO PARA A DIREITA
          if re.findall('\d+', s[start:end]) != []:
            parte_direita_numero.append((start, end))
            break
          if end+k > len(s)-1:break
          frase = s[start:end+k]
          #print("frase 1", frase)
          if frase[-1] == '.' or frase[-1] == ',':break
          if frase[-1].isspace() == False:
            if frase[-1].isdigit() == True and s[start:end+k+1][-1].isdigit() == False:  #dorms 2
              if end+k in indices_preco or end+k in indices_area: break
              parte_direita_numero.append((start, end+k))
            elif frase[-1].isdigit() == True and s[start:end+k+1][-1].isdigit() == True: #dorms 02
              if end+k in indices_preco or end+k in indices_area: break
              parte_direita_numero.append((start, end+k+1))
            break
        for j in range(1,50):
          if start-j < 0:break
          if start-j-1 < 0: j+=1
          frase = s[start-j:end]
          #print("frase2", frase)
          try:
            if frase[0] == '.' or frase[0] == ',':break
          except:break
          if frase[0].isspace() == False:
            if start-j in indices_preco or start-j in indices_area: break
            if frase[0].isdigit() == True and s[start-j-1:end][0].isdigit() == False:  #2 dorms 
              parte_esquerda_numero.append( (start-j, end) )
            if frase[0].isdigit() == True and s[start-j-1:end][0].isdigit() == True:  #02 dorms 
              parte_esquerda_numero.append( (start-j-2, end) )
            break
      #print(parte_direita_numero, parte_esquerda_numero)
      if parte_direita_numero != [] and parte_esquerda_numero != []:
        #s = s.replace(',','').replace('.','')
        inicio = parte_esquerda_numero[0][0]
        inicio_frase = inicio
        fim = parte_esquerda_numero[0][1]
        fim_parte_direita = parte_direita_numero[0][1]
        fim2 = fim_parte_direita
        #print("ref", s[inicio:fim2])
        indice_space, space = inicio, 0
        inicio_frase = -1
        #inicio_frase = inicio
        k = 0
        for _ in range(1,25):   #para trás
          brk = 0
          k +=1
          if inicio-k < 0:break
          if inicio_frase != -1:inicio = inicio_frase
          if s[inicio-k].isspace():space +=1 #CONFERIR SE TEM UMA OUTRA PALAVRA ANTES DO NÚMERO ANTERIOR
          if s[inicio-k]==',' or s[inicio-k] == '.':
            inicio_frase = inicio - k
            break
          if verify_tags_and_numbers(s, s[inicio-k:inicio]) == 1:
            space_ant = 0
            for j in range(1,5):
              if s[inicio-k-j].isdigit():
                inicio_frase = inicio-k-j
                #print("ENCONTROU NÚMERO", s[inicio_frase:fim])
                k = 0
                break
              if s[inicio-k-j].isspace():
                space_ant +=1
              if space_ant == 1 and re.findall('\w', s[inicio-k-j]) != []:
                #print("NÂO TEM NÚMERO ANTES", s[inicio-k:fim])
                inicio_frase = inicio-k
                brk = 1
                break
            if brk == 1:
              break
        fim_frase = -1
        digit_next, space_next, f, fim2 = 0, 0, -1, fim_parte_direita+1
        #print("inicio", s[inicio:fim2])
        for _ in range(1,25): #para frente
          f +=1
          number_with_word = 0
          if fim2+f > len(s) -1:
            #print(fim2+1, len(s))
            break
          if s[fim2+f].isspace():
            space_next +=1
          
          #print("--", s[fim2:fim2+f+1])
          #if ' ' in s[fim2:fim2+f+1]:
          #  if verify_tags_and_numbers(s, s[fim2:fim2+f+1]) == 0:
          #    print(fim_parte_direita, fim2)
          #    while fim_parte_direita >= fim2:
          #      fim2 -=1
          #    print(s[start:end])
          #    while  s[fim2] != s[start:end][-1]: fim2 -=1
          #    print("break aqio")
          #    break
          #if space_next >= 2 and verify_tags_and_numbers(s, s[fim2:fim2+f+1]) == 0:
          #  fim2 -=1
            #print("break aqui")
          #  break

          if space_next == 1 and re.findall('\d', s[fim2:fim2+f+1]) != []:
            #print("aqui", s[fim2:fim2+f+1])
            number_with_word = 1
          if (number_with_word == 1 or s[fim2+f].isdigit() ):#and verify_tags_and_numbers(s, s[fim2:fim2+f+1]) == 1:
            if verify_tags_and_numbers(s, s[fim2:fim2+f+1]) == 1:
              #print("encontrou", s[fim2:fim2+f+1])
              fim2 = fim2+f+1
              f = 0 #carros 3
              space_next = 0
            else:break

        #while re.findall('\w', s[inicio_frase:fim2][-1]) != [] and s[inicio_frase:fim2].strip()[0].isdigit():
        #  fim2 -=1
        #if s[inicio_frase:fim2].strip()[0].isdigit():
        #  space = 0
        #  for p in range(1,20):
        #    if fim2+p+1 > len(s)-1:break
        #    if s[fim2+p].isspace(): space +=1
        #    if space >= 1:
        #      if verify_tags_and_numbers(s, s[fim2:fim2+p+1]) == 0:
        #        fim2 -=1
        #      break
        #print("print(", s[inicio_frase:fim2])
        list_return_frase.append( ( s[inicio_frase:fim2], len(s[inicio_frase:fim2]) ) )
      #break
    #break

  dorms_rgx_only, suites_rgx_only, vagas_rgx_only, banheiro_rgx_only, quartos_rgx_only, garagem_rgx_only, carros_rgx_only = 'NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM'
  if len(list_return_frase) == 0: return dorms_rgx_only, suites_rgx_only, vagas_rgx_only, banheiro_rgx_only, quartos_rgx_only, garagem_rgx_only, carros_rgx_only
  try:
    list_return_frase.sort(reverse=True)
    frase_correct = list_return_frase[0][0].strip()
    #if frase_correct[0] == '':return dorms_rgx_only, suites_rgx_only, vagas_rgx_only, banheiro_rgx_only, quartos_rgx_only, garagem_rgx_only, carros_rgx_only
    #if frase_correct[0][0].isdigit() and frase_correct[0][-1].isdigit():
      #frase_correct = frase_correct[1:].strip()
  except:return dorms_rgx_only, suites_rgx_only, vagas_rgx_only, banheiro_rgx_only, quartos_rgx_only, garagem_rgx_only, carros_rgx_only
  #print(frase_correct)
  try:
    if frase_correct[0].isdigit():
      if (re.findall('\d{1,2}\s*dorm', frase_correct)) != []:
        dorms_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*dorm', frase_correct)[0])[0]
      if (re.findall('\d{1,2}\s*suit', frase_correct)) != []:
        suites_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*suit', frase_correct)[0])[0]
      if (re.findall('\d{1,2}\s*vaga', frase_correct)) != []:
        vagas_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*vaga', frase_correct)[0])[0]
      if (re.findall('\d{1,2}\s*banheiro', frase_correct)) != []:
        banheiro_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*banheiro', frase_correct)[0])[0]
      if (re.findall('\d{1,2}\s*quarto', frase_correct)) != []:
        quartos_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*quarto', frase_correct)[0])[0]
      if (re.findall('\d{1,2}\s*garage', frase_correct)) != []:
        garagem_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*garage', frase_correct)[0])[0]
      if (re.findall('\d{1,2}\s*carro', frase_correct)) != []:
        carros_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*carro', frase_correct)[0])[0]
    else:
      if (re.findall('dorm\w*\s*\d{1,2}', frase_correct)) != []:
        dorms_rgx_only = re.findall('\d{1,2}', re.findall('dorm\w*\s*\d{1,2}', frase_correct)[0])[0]
      if (re.findall('suit\w*\s*\d{1,2}', frase_correct)) != []:
        suites_rgx_only = re.findall('\d{1,2}', re.findall('suit\w*\s*\d{1,2}', frase_correct)[0])[0]
      if (re.findall('vaga\w*\s*\d{1,2}', frase_correct)) != []:
        vagas_rgx_only = re.findall('\d{1,2}', re.findall('vaga\w*\s*\d{1,2}', frase_correct)[0])[0]
      if (re.findall('banheiro\w*\s*\d{1,2}', frase_correct)) != []:
        banheiro_rgx_only = re.findall('\d{1,2}', re.findall('banheiro\w*\s*\d{1,2}', frase_correct)[0])[0]
      if (re.findall('quarto\w*\s*\d{1,2}', frase_correct)) != []:
        quartos_rgx_only = re.findall('\d{1,2}', re.findall('quarto\w*\s*\d{1,2}', frase_correct)[0])[0]
      if (re.findall('garage\w*\s*\d{1,2}', frase_correct)) != []:
        garagem_rgx_only = re.findall('\d{1,2}', re.findall('garage\w*\s*\d{1,2}', frase_correct)[0])[0]
      if (re.findall('carro\w*\s*\d{1,2}', frase_correct)) != []:
        carros_rgx_only = re.findall('\d{1,2}', re.findall('carro\w*\s*\d{1,2}', frase_correct)[0])[0]
  except:pass
  return dorms_rgx_only, suites_rgx_only, vagas_rgx_only, banheiro_rgx_only, quartos_rgx_only, garagem_rgx_only, carros_rgx_only

In [ ]:
def comodosChavesJuntos(s):
  indices_preco, indices_area = [], []
  s = re.sub('\s+', ' ', unidecode(s.lower().replace('(','').replace(')','').replace(':','').replace('\xa0','')))
  list_tags_search = ['dorm', 'suit', 'vaga', 'banheiro', 'quarto', 'garage', 'carro', 'box']
  list_return_frase = []
  doc = nlp8(s) 
  indices_preco = indicesPreco(s)
  indices_area = indicesArea(s)
  for tag in list_tags_search:
    regex_tag = tag+'\w*'
    number_direita, number_esquerda = [], []
    for match in re.finditer(regex_tag, doc.text):
      start, end = match.span()
      span = doc.char_span(start, end, label=tag)
      parte_direita_numero, parte_esquerda_numero = [], []
      if span:
        #print("SPAN", span)
        for k in range(1,50):                           #NÚMERO PARA A DIREITA
          if re.findall('\d+', s[start:end]) != []:
            parte_direita_numero.append((start, end))
            break
          if end+k > len(s)-1:break
          frase = s[start:end+k]
          #print("frase 1", frase)
          if frase[-1] == '.' or frase[-1] == ',':break
          if frase[-1].isspace() == False:
            if frase[-1].isdigit() == True and s[start:end+k+1][-1].isdigit() == False:  #dorms 2
              if end+k in indices_preco or end+k in indices_area: break
              parte_direita_numero.append((start, end+k))
            elif frase[-1].isdigit() == True and s[start:end+k+1][-1].isdigit() == True: #dorms 02
              if end+k in indices_preco or end+k in indices_area: break
              parte_direita_numero.append((start, end+k+1))
            break
        for j in range(1,50):
          if start-j < 0:break
          if start-j-1 < 0: j+=1
          frase = s[start-j:end]
          #print("frase2", frase)
          try:
            if frase[0] == '.' or frase[0] == ',':break
          except:break
          if frase[0].isspace() == False:
            if start-j in indices_preco or start-j in indices_area: break
            if frase[0].isdigit() == True and s[start-j-1:end][0].isdigit() == False:  #2 dorms 
              parte_esquerda_numero.append( (start-j, end) )
            if frase[0].isdigit() == True and s[start-j-1:end][0].isdigit() == True:  #02 dorms 
              parte_esquerda_numero.append( (start-j-2, end) )
            break
      #print(parte_direita_numero, parte_esquerda_numero)
      if parte_direita_numero != [] and parte_esquerda_numero != []:
        #s = s.replace(',','').replace('.','')
        inicio = parte_esquerda_numero[0][0]
        inicio_frase = inicio
        fim = parte_esquerda_numero[0][1]
        fim_parte_direita = parte_direita_numero[0][1]
        fim2 = fim_parte_direita
        #print("ref", s[inicio:fim2])
        indice_space, space = inicio, 0
        inicio_frase = -1
        #inicio_frase = inicio
        k = 0
        for _ in range(1,25):   #para trás
          brk = 0
          k +=1
          if inicio-k < 0:break
          if inicio_frase != -1:inicio = inicio_frase
          if s[inicio-k].isspace():space +=1 #CONFERIR SE TEM UMA OUTRA PALAVRA ANTES DO NÚMERO ANTERIOR
          if s[inicio-k]==',' or s[inicio-k] == '.':
            inicio_frase = inicio - k
            break
          if verify_tags_and_numbers(s, s[inicio-k:inicio]) == 1:
            space_ant = 0
            for j in range(1,5):
              if s[inicio-k-j].isdigit():
                inicio_frase = inicio-k-j
                #print("ENCONTROU NÚMERO", s[inicio_frase:fim])
                k = 0
                break
              if s[inicio-k-j].isspace():
                space_ant +=1
              if space_ant == 1 and re.findall('\w', s[inicio-k-j]) != []:
                #print("NÂO TEM NÚMERO ANTES", s[inicio-k:fim])
                inicio_frase = inicio-k
                brk = 1
                break
            if brk == 1:
              break
        fim_frase = -1
        digit_next, space_next, f, fim2 = 0, 0, -1, fim_parte_direita+1
        #print("inicio", s[inicio:fim2])
        for _ in range(1,25): #para frente
          f +=1
          number_with_word = 0
          if fim2+f > len(s) -1:
            #print(fim2+1, len(s))
            break
          if s[fim2+f].isspace():
            space_next +=1
          
          #print("--", s[fim2:fim2+f+1])
          #if ' ' in s[fim2:fim2+f+1]:
          #  if verify_tags_and_numbers(s, s[fim2:fim2+f+1]) == 0:
          #    print(fim_parte_direita, fim2)
          #    while fim_parte_direita >= fim2:
          #      fim2 -=1
          #    print(s[start:end])
          #    while  s[fim2] != s[start:end][-1]: fim2 -=1
          #    print("break aqio")
          #    break
          #if space_next >= 2 and verify_tags_and_numbers(s, s[fim2:fim2+f+1]) == 0:
          #  fim2 -=1
            #print("break aqui")
          #  break

          if space_next == 1 and re.findall('\d', s[fim2:fim2+f+1]) != []:
            #print("aqui", s[fim2:fim2+f+1])
            number_with_word = 1
          if (number_with_word == 1 or s[fim2+f].isdigit() ):#and verify_tags_and_numbers(s, s[fim2:fim2+f+1]) == 1:
            if verify_tags_and_numbers(s, s[fim2:fim2+f+1]) == 1:
              #print("encontrou", s[fim2:fim2+f+1])
              fim2 = fim2+f+1
              f = 0 #carros 3
              space_next = 0
            else:break

        #while re.findall('\w', s[inicio_frase:fim2][-1]) != [] and s[inicio_frase:fim2].strip()[0].isdigit():
        #  fim2 -=1
        #if s[inicio_frase:fim2].strip()[0].isdigit():
        #  space = 0
        #  for p in range(1,20):
        #    if fim2+p+1 > len(s)-1:break
        #    if s[fim2+p].isspace(): space +=1
        #    if space >= 1:
        #      if verify_tags_and_numbers(s, s[fim2:fim2+p+1]) == 0:
        #        fim2 -=1
        #      break
        #print("print(", s[inicio_frase:fim2])
        list_return_frase.append( ( s[inicio_frase:fim2], len(s[inicio_frase:fim2]) ) )
      #break
    #break

  dorms_rgx_only, suites_rgx_only, vagas_rgx_only, banheiro_rgx_only, quartos_rgx_only, garagem_rgx_only, carros_rgx_only = 'NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM'
  if len(list_return_frase) == 0: return dorms_rgx_only, suites_rgx_only, vagas_rgx_only, banheiro_rgx_only, quartos_rgx_only, garagem_rgx_only, carros_rgx_only
  try:
    list_return_frase.sort(reverse=True)
    frase_correct = list_return_frase[0][0].strip()
    #if frase_correct[0] == '':return dorms_rgx_only, suites_rgx_only, vagas_rgx_only, banheiro_rgx_only, quartos_rgx_only, garagem_rgx_only, carros_rgx_only
    #if frase_correct[0][0].isdigit() and frase_correct[0][-1].isdigit():
      #frase_correct = frase_correct[1:].strip()
  except:return dorms_rgx_only, suites_rgx_only, vagas_rgx_only, banheiro_rgx_only, quartos_rgx_only, garagem_rgx_only, carros_rgx_only
  #print(frase_correct)
  try:
    if frase_correct[0].isdigit():
      if (re.findall('\d{1,2}\s*dorm', frase_correct)) != []:
        dorms_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*dorm', frase_correct)[0])[0]
      if (re.findall('\d{1,2}\s*suit', frase_correct)) != []:
        suites_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*suit', frase_correct)[0])[0]
      if (re.findall('\d{1,2}\s*vaga', frase_correct)) != []:
        vagas_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*vaga', frase_correct)[0])[0]
      if (re.findall('\d{1,2}\s*banheiro', frase_correct)) != []:
        banheiro_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*banheiro', frase_correct)[0])[0]
      if (re.findall('\d{1,2}\s*quarto', frase_correct)) != []:
        quartos_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*quarto', frase_correct)[0])[0]
      if (re.findall('\d{1,2}\s*garage', frase_correct)) != []:
        garagem_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*garage', frase_correct)[0])[0]
      if (re.findall('\d{1,2}\s*carro', frase_correct)) != []:
        carros_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*carro', frase_correct)[0])[0]
    else:
      if (re.findall('dorm\w*\s*\d{1,2}', frase_correct)) != []:
        dorms_rgx_only = re.findall('\d{1,2}', re.findall('dorm\w*\s*\d{1,2}', frase_correct)[0])[0]
      if (re.findall('suit\w*\s*\d{1,2}', frase_correct)) != []:
        suites_rgx_only = re.findall('\d{1,2}', re.findall('suit\w*\s*\d{1,2}', frase_correct)[0])[0]
      if (re.findall('vaga\w*\s*\d{1,2}', frase_correct)) != []:
        vagas_rgx_only = re.findall('\d{1,2}', re.findall('vaga\w*\s*\d{1,2}', frase_correct)[0])[0]
      if (re.findall('banheiro\w*\s*\d{1,2}', frase_correct)) != []:
        banheiro_rgx_only = re.findall('\d{1,2}', re.findall('banheiro\w*\s*\d{1,2}', frase_correct)[0])[0]
      if (re.findall('quarto\w*\s*\d{1,2}', frase_correct)) != []:
        quartos_rgx_only = re.findall('\d{1,2}', re.findall('quarto\w*\s*\d{1,2}', frase_correct)[0])[0]
      if (re.findall('garage\w*\s*\d{1,2}', frase_correct)) != []:
        garagem_rgx_only = re.findall('\d{1,2}', re.findall('garage\w*\s*\d{1,2}', frase_correct)[0])[0]
      if (re.findall('carro\w*\s*\d{1,2}', frase_correct)) != []:
        carros_rgx_only = re.findall('\d{1,2}', re.findall('carro\w*\s*\d{1,2}', frase_correct)[0])[0]
  except:pass
  return dorms_rgx_only, suites_rgx_only, vagas_rgx_only, banheiro_rgx_only, quartos_rgx_only, garagem_rgx_only, carros_rgx_only

In [ ]:
#sentença extraída normal do list_ner = com espaço -> verificar qual número é mais próximo do cômodo
def returnTagsContSpace(s):
  s = unidecode(s.lower().replace('(','').replace(')','').replace(':','').replace('\xa0',''))
  doc = nlp8(s)
  dorm_rgx_tag_space, suite_rgx_tag_space, vaga_rgx_tag_space, banheiro_rgx_tag_space, quarto_rgx_tag_space, garagem_rgx_tag_space, carro_rgx_tag_space = 'NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM'
  for tag in ['dorm', 'suit', 'vaga', 'banheiro', 'quarto', 'garage', 'carro']:
    regex_tag = tag+'\w*'
    list_values_rigth, list_values_left = [], []
    for match in re.finditer(regex_tag, doc.text):
      start, end = match.span()
      span = doc.char_span(start, end, label=tag)
      if span:
        for i in range(1,100): #para a direita
          string_frase = s[start:end+i+1]
          if end+i >= len(s):break
          if s[end+i].isdigit() and s[end+i+1].isdigit() == True:
            #if end+i+1 <= len(s)-1:#.isdigit() == True: #vagas 50
              #if s[end+i+1].isdigit() == True:
                string_frase = s[start:end+i+2]
                list_values_rigth.append(string_frase)
                break
          if s[end+i].isdigit():
            string_frase = s[start:end+i+1]
            list_values_rigth.append(string_frase)
            break
          #elif s[end+i].isdigit() and s[end+i+1].isdigit() == False: #vagas 5
          elif s[end+i].isdigit() and s[end+i+1].isdigit() == False:
            #if end+i+1 <= len(s)-1:#.isdigit() == True:
              #if s[end+i+1].isdigit() == False:
                string_frase = s[start:end+i+1]
                list_values_rigth.append(string_frase)
                break
          elif s[end+i].isdigit() == False and s[end+i].isspace() == False: #quer dizer que é uma letra
            #string_frase = s[start:end+i+1]
            break
        for i in range(1,100): #para a esquerda
          if s[start-i].isdigit():
            string_frase = s[start-i:end]
            if s[start-i-1] != 'm' and s[start-i-1].isdigit() == False: #6 quartos
              list_values_left.append(string_frase)
              break
            elif s[start-i-1] != 'm' and s[start-i-1].isdigit() == True: #36 quartos
              list_values_left.append(s[start-i-1:end])
              break
          elif s[start-i].isdigit() == False and s[start-i].isspace() == False: #quer dizer que é uma letra
            break
    list_cont_space = []
    list_values_rigth.extend(list_values_left)
    #print(list_values_rigth)
    if list_values_rigth != []:
      for v in list_values_rigth:
        list_cont_space.append( (len(re.findall('\s', v)), v))

      list_cont_space.sort(key=lambda x:x[0])
      if len(list_cont_space) > 1:
        if list_cont_space[0][0] == list_cont_space[1][0] and re.findall('\d+', list_cont_space[0][1]) != re.findall('\d+', list_cont_space[1][1]):
          #return dorm_rgx_tag_space, suite_rgx_tag_space, vaga_rgx_tag_space, banheiro_rgx_tag_space, quarto_rgx_tag_space, garagem_rgx_tag_space, carro_rgx_tag_space        
          continue
      
      if tag == 'dorm':dorm_rgx_tag_space = int(re.findall('\d{1,2}', list_cont_space[0][1])[0])
      elif tag == 'suit':suite_rgx_tag_space = int(re.findall('\d{1,2}', list_cont_space[0][1])[0])
      elif tag == 'vaga':vaga_rgx_tag_space = int(re.findall('\d{1,2}', list_cont_space[0][1])[0])
      elif tag == 'banheiro':banheiro_rgx_tag_space = int(re.findall('\d{1,2}', list_cont_space[0][1])[0])
      elif tag == 'quarto':quarto_rgx_tag_space = int(re.findall('\d{1,2}', list_cont_space[0][1])[0])
      elif tag == 'garage':garagem_rgx_tag_space = int(re.findall('\d{1,2}', list_cont_space[0][1])[0])
      elif tag == 'carro':carro_rgx_tag_space = int(re.findall('\d{1,2}', list_cont_space[0][1])[0])
      
  return dorm_rgx_tag_space, suite_rgx_tag_space, vaga_rgx_tag_space, banheiro_rgx_tag_space, quarto_rgx_tag_space, garagem_rgx_tag_space, carro_rgx_tag_space

In [ ]:
def returnAreaTerreno(url, s):
  s = unidecode(re.sub('\s+', ' ', s.lower().replace('(','').replace(')','').replace(':','').replace('\xa0','')))
  doc = nlp8(s.lower())
  expression = re.compile('terreno')
  list_values_terreno = []
  for match in re.finditer(expression, doc.text):
    start, end = match.span()
    span = doc.char_span(start, end, label='terreno')
    if span:
      ok2 = 0
      for i in range(1,20):
        string_verify = s[start:end+i][-2:]
        if (string_verify == 'm2' or string_verify == 'm²' or s[start:end+i][-3:] == 'm ²' or s[start:end+i][-3:] == 'm 2') and (re.findall('\d', s[start:end+i][-4]) != []):
          frase_select = s[start:end+i]
          if ('area' in frase_select or 'total' in frase_select or 'terreno' in frase_select) and ('priv' not in frase_select):
            for v in re.findall('\d+[,.]?\d+',s[start:end+i] ):
              list_values_terreno.append(v)
            break
          break
      for i in range(1,30):
        string_verify = s[start-i:end][:2]
        if string_verify == 'm2' or string_verify == 'm²' or s[start-i:end][:3] == 'm ²' or s[start-i:end][:3] == 'm 2':
          not_frase = 0
          contn, cont_space = 0, 0
          frase_select2 = s[start-i:end]
          for k in range(1,30):
            frase_select2 = s[start-i-k:end]
            frasev = s[start-i-k]
            if re.findall('\d', frasev) != []:
              contn +=1
            elif re.findall('\s', frasev) != [] and frasev[0:3] != 'com' and frasev[0:2] != 'de':
              cont_space +=1
            if cont_space > 1: 
              if ('priv' in frase_select2 or 'box' in frase_select2):
                not_frase = 1
                break
              if not_frase == 0:
                for v in re.findall('\d+[,.]?\d+',s[start-i-k:end] ):
                  list_values_terreno.append(v)
              break
          break
  return list_values_terreno

In [ ]:
def returnArea(url, s):
  #print(url)
  s = unidecode(re.sub('\s+', ' ', s.lower().replace('(','').replace(')','').replace(':','').replace('\xa0','')))
  doc = nlp8(s.lower())
  expression = re.compile('area')
  list_values_area = []
  parte_1_direita = False
  for match in re.finditer(expression, doc.text):
    start, end = match.span()
    span = doc.char_span(start, end, label='area')
    if span:
      for i in range(1,20):
        string_verify = s[start:end+i][-2:]
        if (string_verify == 'm2' or string_verify == 'm²' or s[start:end+i][-3:] == 'm ²' or s[start:end+i][-3:] == 'm 2') and (re.findall('\d', s[start:end+i][-4]) != []):
          frase_select = s[start:end+i]
          if ('area' in frase_select or 'total' in frase_select or 'terreno' in frase_select) and ('privativ' not in frase_select):
            #print(s[start:end+i])
            for v in re.findall('\d+[,.]?\d+',s[start:end+i] ):
              list_values_area.append(v)
            parte_1_direita = True
            break
      for i in range(1,20):
        string_verify = s[start-i:end][:2]
        if string_verify == 'm2' or string_verify == 'm²' or s[start-i:end][:3] == 'm ²' or s[start-i:end][:3] == 'm 2':
          not_frase = 0
          contn, cont_space = 0, 0
          frase_select2 = s[start-i:end]
          for k in range(1,30):
            #if (re.findall('[a-z]', s[start-i-k]) != []) and (s[start-i-k] !='.' or s[start-i-k] != ','):
            #print(s[start-i-k:end])
            frase_select2 = s[start-i-k:end]
            frasev = s[start-i-k]
            if re.findall('\d', frasev) != []:
              contn +=1
            elif re.findall('\s', frasev) != [] and frasev[0:3] != 'com' and frasev[0:2] != 'de':
              cont_space +=1
            if cont_space > 1: 
              #print("FRASE", frase_select2)
              if ('priv' in frase_select2 or 'box' in frase_select2) and (parte_1_direita == True):
                not_frase = 1
                break
              if not_frase == 0:
                for v in re.findall('\d+[,.]?\d+',s[start-i-k:end] ):
                  list_values_area.append(v)
              break
          break
    #print(list_values_area)
  return list_values_area




def returnAreaPrivativa(url, s):
  s = unidecode(re.sub('\s+', ' ', s.lower().replace('(','').replace(')','').replace(':','').replace('\xa0','')))
  doc = nlp8(s.lower())
  expression = re.compile('privativ\w*')
  list_values_area_priv = []
  for match in re.finditer(expression, doc.text):
    start, end = match.span()
    span = doc.char_span(start, end, label='priv')
    if span:
      ok2 = 0
      for i in range(1,20):
        string_verify = s[start:end+i][-2:]
        if (string_verify == 'm2' or string_verify == 'm²' or s[start:end+i][-3:] == 'm ²' or s[start:end+i][-3:] == 'm 2') and (re.findall('\d', s[start:end+i][-4]) != []):
          for v in re.findall('\d+[,.]?\d+',s[start:end+i] ):
            list_values_area_priv.append(v)
          ok2 +=1
          break
      #CONFERE PARA A ESQUERDA SE TEM OUTRO NÚMERO
      for i in range(1,20):
        string_verify = s[start-i:end][:2]
        if string_verify == 'm2' or string_verify == 'm²' or s[start-i:end][:3] == 'm ²' or s[start-i:end][:3] == 'm 2':
          not_frase = 0
          contn, cont_space = 0, 0
          frase_select2 = s[start-i:end]
          for k in range(1,30):
            frase_select2 = s[start-i-k:end]
            frasev = s[start-i-k]
            if re.findall('\d', frasev) != []:
              contn +=1
            elif re.findall('\s', frasev) != [] and frasev[0:3] != 'com' and frasev[0:2] != 'de':
              cont_space +=1
            if cont_space > 1: 
              if ('terreno' in frase_select2 or 'box' in frase_select2 or 'total' in frase_select2): #PARA DEIXAR MAIS FLEXÍVEL E ADICIONAR POSSÍVEIS MAIS VALORES -> ok2 == 0
                not_frase = 1
                break
              if not_frase == 0:
                for v in re.findall('\d+[,.]?\d+',s[start-i-k:end] ):
                  #print("insere", v)
                  list_values_area_priv.append(v)
              break
          break
  return list_values_area_priv

In [ ]:
def returnArea(url, s):
  #print(url)
  s = unidecode(re.sub('\s+', ' ', s.lower().replace('(','').replace(')','').replace(':','').replace('\xa0','')))
  doc = nlp8(s.lower())
  expression = re.compile('area')
  list_values_area = []
  parte_1_direita = False
  for match in re.finditer(expression, doc.text):
    start, end = match.span()
    span = doc.char_span(start, end, label='area')
    if span:
      for i in range(1,20):
        string_verify = s[start:end+i][-2:]
        if (string_verify == 'm2' or string_verify == 'm²' or s[start:end+i][-3:] == 'm ²' or s[start:end+i][-3:] == 'm 2') and (re.findall('\d', s[start:end+i][-4]) != []):
          frase_select = s[start:end+i]
          if ('area' in frase_select or 'total' in frase_select or 'terreno' in frase_select) and ('privativ' not in frase_select):
            #print(s[start:end+i])
            for v in re.findall('\d+[,.]?\d+',s[start:end+i] ):
              list_values_area.append(v)
            parte_1_direita = True
            break
      for i in range(1,20):
        string_verify = s[start-i:end][:2]
        if string_verify == 'm2' or string_verify == 'm²' or s[start-i:end][:3] == 'm ²' or s[start-i:end][:3] == 'm 2':
          not_frase = 0
          contn, cont_space = 0, 0
          frase_select2 = s[start-i:end]
          for k in range(1,30):
            #if (re.findall('[a-z]', s[start-i-k]) != []) and (s[start-i-k] !='.' or s[start-i-k] != ','):
            #print(s[start-i-k:end])
            frase_select2 = s[start-i-k:end]
            frasev = s[start-i-k]
            if re.findall('\d', frasev) != []:
              contn +=1
            elif re.findall('\s', frasev) != [] and frasev[0:3] != 'com' and frasev[0:2] != 'de':
              cont_space +=1
            if cont_space > 1: 
              #print("FRASE", frase_select2)
              if ('priv' in frase_select2 or 'box' in frase_select2) and (parte_1_direita == True):
                not_frase = 1
                break
              if not_frase == 0:
                for v in re.findall('\d+[,.]?\d+',s[start-i-k:end] ):
                  list_values_area.append(v)
              break
          break
    #print(list_values_area)
  return list_values_area

In [ ]:
def converteStringToDigito(string):
  list_number_per_extenso =  ['zero', 'um', 'dois', 'tres', 'quatro', 'cinco', 'seis', 'sete', 'oito', 'nove', 'dez',
           'onze', 'doze', 'treze', 'catorze', 'quinze', 'dezesseis', 'dezessete', 'dezoito', 'dezenove', 'vinte']
  
  for i in range(1,len(list_number_per_extenso)):
    if list_number_per_extenso[i] == string:
      return i


def returnComodosOk(list_frases):
  dorms_rgx_only, suites_rgx_only, vagas_rgx_only, banheiro_rgx_only, quartos_rgx_only, garagem_rgx_only, carros_rgx_only = 'NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM'
  list_frases.sort(key = lambda x: x[1], reverse=True)
  list_frase_correct = []
  for fr in list_frases:
    dorms_rgx_only, suites_rgx_only, vagas_rgx_only, banheiro_rgx_only, quartos_rgx_only, garagem_rgx_only, carros_rgx_only = 'NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM'
    correct  = 0
    fr = fr[0].strip()
    frase_correct = fr
    if len(fr) == 0 or fr == None:break
    #print(fr)
    if fr[-1].isdigit() == False:
      frase_correct = fr
      if (re.findall('\d{1,2}\s*dorm', frase_correct)) != []:
        dorms_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*dorm', frase_correct)[0])[0]
        correct +=1
      if (re.findall('\d{1,2}\s*suit', frase_correct)) != []:
        suites_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*suit', frase_correct)[0])[0]
        correct +=1
      if (re.findall('\d{1,2}\s*vaga', frase_correct)) != []:
        vagas_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*vaga', frase_correct)[0])[0]
        correct +=1
      if (re.findall('\d{1,2}\s*banheiro', frase_correct)) != []:
        banheiro_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*banheiro', frase_correct)[0])[0]
        correct +=1
      if (re.findall('\d{1,2}\s*quarto', frase_correct)) != []:
        quartos_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*quarto', frase_correct)[0])[0]
        correct +=1
      if (re.findall('\d{1,2}\s*garage', frase_correct)) != []:
        garagem_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*garage', frase_correct)[0])[0]
        correct +=1
      if (re.findall('\d{1,2}\s*carro', frase_correct)) != []:
        carros_rgx_only = re.findall('\d{1,2}', re.findall('\d{1,2}\s*carro', frase_correct)[0])[0]
        correct +=1
    elif fr[-1].isdigit():
        #frase_correct = fr[fr.find(' '):]
        frase_correct = fr
        if (re.findall('dorm\w*\s*\d{1,2}', frase_correct)) != []:
          dorms_rgx_only = re.findall('\d{1,2}', re.findall('dorm\w*\s*\d{1,2}', frase_correct)[0])[0]
          correct +=1
        if (re.findall('suit\w*\s*\d{1,2}', frase_correct)) != []:
          suites_rgx_only = re.findall('\d{1,2}', re.findall('suit\w*\s*\d{1,2}', frase_correct)[0])[0]
          correct +=1
        if (re.findall('vaga\w*\s*\d{1,2}', frase_correct)) != []:
          vagas_rgx_only = re.findall('\d{1,2}', re.findall('vaga\w*\s*\d{1,2}', frase_correct)[0])[0]
          correct +=1
        if (re.findall('banheiro\w*\s*\d{1,2}', frase_correct)) != []:
          banheiro_rgx_only = re.findall('\d{1,2}', re.findall('banheiro\w*\s*\d{1,2}', frase_correct)[0])[0]
          correct +=1
        if (re.findall('quarto\w*\s*\d{1,2}', frase_correct)) != []:
          quartos_rgx_only = re.findall('\d{1,2}', re.findall('quarto\w*\s*\d{1,2}', frase_correct)[0])[0]
          correct +=1
        if (re.findall('garage\w*\s*\d{1,2}', frase_correct)) != []:
          garagem_rgx_only = re.findall('\d{1,2}', re.findall('garage\w*\s*\d{1,2}', frase_correct)[0])[0]
          correct +=1
        if (re.findall('carro\w*\s*\d{1,2}', frase_correct)) != []:
          carros_rgx_only = re.findall('\d{1,2}', re.findall('carro\w*\s*\d{1,2}', frase_correct)[0])[0]
          correct +=1
    if correct > 1:
      #print(frase_correct)
      list_frase_correct.append((frase_correct, correct, dorms_rgx_only, suites_rgx_only, vagas_rgx_only, banheiro_rgx_only, quartos_rgx_only, garagem_rgx_only, carros_rgx_only))
  if len(list_frase_correct) > 1:
    #print(list_frase_correct)
    list_frase_correct.sort(key = lambda x: x[1], reverse=True)
    if list_frase_correct[0][1] == list_frase_correct[1][1]:
      if len(list_frase_correct[0][0]) > len(list_frase_correct[1][0]):
        list_frase_correct[0], list_frase_correct[1] = list_frase_correct[1], list_frase_correct[0]
  #print("Frase", list_frase_correct)
  if len(list_frase_correct) > 0:
    dorms_rgx_only, suites_rgx_only, vagas_rgx_only = list_frase_correct[0][2],list_frase_correct[0][3],list_frase_correct[0][4],
    banheiro_rgx_only, quartos_rgx_only, garagem_rgx_only, carros_rgx_only = list_frase_correct[0][5],list_frase_correct[0][6],list_frase_correct[0][7],list_frase_correct[0][8]
  return dorms_rgx_only, suites_rgx_only, vagas_rgx_only, banheiro_rgx_only, quartos_rgx_only, garagem_rgx_only, carros_rgx_only

In [ ]:
def verify_tag_entry(word, tipo = 'tag'):
  if tipo == 'tag': list_search = ['dorm', 'suit', 'vaga', 'banheiro', 'quarto', 'garage', 'carro', 'box']  
  elif tipo == 'number_extense': list_search = ['zero', 'um', 'dois', 'tres', 'quatro', 'cinco', 'seis', 'sete', 'oito', 'nove', 'dez',
           'onze', 'doze', 'treze', 'catorze', 'quinze', 'dezesseis', 'dezessete', 'dezoito', 'dezenove', 'vinte']
  for tag in list_search:
    if tag in word: 
      #print("word", word, "ok")
      return 1
  return 0

list_number_per_extenso =  ['zero', 'um', 'dois', 'tres', 'quatro', 'cinco', 'seis', 'sete', 'oito', 'nove', 'dez',
           'onze', 'doze', 'treze', 'catorze', 'quinze', 'dezesseis', 'dezessete', 'dezoito', 'dezenove', 'vinte']

def comodosJuntosNEW(s):
  s = re.sub('\s+', ' ', unidecode(s.lower().replace('(','').replace(')','').replace(':','').replace('\xa0','').replace('sendo', '').replace('demi','').replace('demi-','')).replace('ate', ''))
  doc = nlp8(s)
  pontuaction = """!"#$%&'()*+, -./:;<=>?@[\]^_`{|}~"""
  list_frases = []
  indices_area = indicesArea(s)
  indices_preco = indicesPreco(s)
  for tag in ['dorm', 'suit', 'vaga', 'banheiro', 'quarto', 'garage', 'carro', 'box']:
    tag_regex = tag+'\w*'
    for match in re.finditer(tag_regex, doc.text):
      start, end = match.span()
      span = doc.char_span(start, end, label='area')
      if span:
        #print("-----SPAN-----", span)
        #confere se tem pontuação para o lado direito
        inicio, fim = start, end
        if s[start:end+1][-1] in pontuaction or s[start:end+2][-1] in pontuaction:
            #print(s[start:end+1])
            #se tem pontuação, então confere para a esquerda se tem um número
            space = 0
            for j in range(1,10):
              if s[start-j].isspace():space +=1
              if space == 2:break
              if s[start-j].isdigit():
                inicio, fim = start-j, end
                #print(s[start-j:end])
        #confere para a ESQUERDA se tem outras tags procuradas
        space = 0
        inicio2 = inicio
        inicio_final = -1
        inicio_aux = -1
        digit_aux = 0
        list_ordens = []
        for e in range(1,100):
          if inicio-e < 0:
            break
          if inicio-e in indices_area or inicio-e in indices_preco:
            inicio_final = inicio - e + 1
            break
          if s[inicio-e].isspace():
            space +=1
          if space == 1:
            #if verify_tag_entry(s[inicio-e+digit_aux:inicio2], tipo = 'tag') == 1:
            ##  print("OKKK")
            inicio_aux = inicio-e
          if s[inicio-e] == ',' or s[inicio-e] == ';' or s[inicio-e] == '|' or s[inicio-e] == '-':
            inicio_final = inicio-e + 1
            break
          elif s[inicio-e].isdigit():
              #if s[inicio-e-1].isdigit():digit_aux -= 1
              if len(list_ordens ) > 0: #verificar também se é de 2 dígitos, como 02 dois dorms
                if s[inicio-e-1].isdigit() == False:
                  if str(converteStringToDigito(list_ordens[-1].strip())) != s[inicio-e]:
                    #print("NÃO")
                    inicio_final = inicio - e + 2
                    break
                elif s[inicio-e-1].isdigit() == True:
                  if str(converteStringToDigito(list_ordens[-1].strip())) != s[inicio-e-1]:
                    inicio_final = inicio - e + 2
                    break
              space = 0
          if space >=2:
            list_ordens = []
            if verify_tag_entry(s[inicio-e+digit_aux:inicio2], tipo = 'tag') == 1 :#or verify_tag_entry(s[inicio-e+digit_aux:inicio2], tipo = 'number_extense') == 1:
              #print("DENTROOOOOOOOOOOOO", s[inicio-e+digit_aux:fim])
              space = 1
              inicio2 = inicio - e + digit_aux
              continue
            elif verify_tag_entry(s[inicio-e+digit_aux:inicio2], tipo = 'number_extense') == 1:
              list_ordens.append(s[inicio-e+digit_aux:inicio2])
              space = 1
              inicio2 = inicio - e + digit_aux
              continue
            else: 
              #number_extense_ver = 0
              #se não tem uma palavra anterior que é igual ou parecida com as tags procuradas, então sai fora:
              #print("TESTEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE", s[inicio-e+digit_aux-5:inicio2],s[inicio-e+digit_aux:inicio2][-1], "ANTES", s[inicio-e+digit_aux-1:inicio2][0])
              if s[inicio-e+digit_aux:inicio2][-1].isdigit():
                inicio_final = inicio2 - 1 + digit_aux
                #print("DIGIT")
                if s[inicio-e+digit_aux-1].isdigit():
                  inicio_final -= 1
              else:inicio_final = inicio2 + digit_aux
              #print("FORAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA", s[inicio-e+digit_aux:inicio2])
              break
          #print(s[inicio-e:fim])
        if s[inicio_final-1].isdigit():inicio_final -=1
        #print("final", s[inicio_final:fim])

        #PARA A DIREITAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA ------------------------------

        fim2 = fim
        space = 0
        fim_final = -1
        fim_aux = -1
        digit_aux = 0
        list_ordens = []
        #print("DIREITAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA", span)
        for d in range(1,100):
          if fim2+ d + 1 > len(s) - 1:
            break
          if fim2 + d in indices_area or fim2 + d in indices_preco:
            fim_final = fim2 + d #+ 1
            break
          if s[fim2 + d].isspace():
            space +=1
          if space == 1:
            fim_aux = fim2 + d + 1
          #print(s[fim2:fim2+d+1])
          if s[fim2+ d ] == ',' or s[fim2+ d ] == ';' or s[fim2+ d ] == '|' or s[fim2+ d ] == '-':
            fim_final = fim2 + d + 1 
            break
          elif s[fim2 + d].isdigit():
              #if s[fim + d + 1-1].isdigit():digit_aux -= 1
              if len(list_ordens) > 0: #verificar também se é de 2 dígitos, como 02 dois dorms
                if s[fim2+d+1].isdigit() == False:
                  if str(converteStringToDigito(list_ordens[-1].strip())) != s[fim2 + d]:
                    fim_final = fim2 + d
                    break
                elif s[fim2+d+1].isdigit() == True:
                  if str(converteStringToDigito(list_ordens[-1].strip())) != s[fim2 + d + 1]:
                    fim_final = fim2 + d + 1
                    break
              space = 2
              fim2 +=d
          if space >=2:
            list_ordens = []
            if verify_tag_entry(s[fim2:fim2+d+1], tipo = 'tag') == 1 :#or verify_tag_entry(s[fim2+d+1:fim2], tipo = 'number_extense') == 1:
              #print("DENTROOOOOOOOOOOOO", s[fim2:fim2+d+1])
              space = 1
              fim2 = fim2 + d +1
              continue
            elif verify_tag_entry(s[fim2:fim2+d+1], tipo = 'number_extense') == 1:
              list_ordens.append(s[fim2:fim2+d+1])
              space = 1
              fim2 = fim2 + d + 1
              continue
            else: 
              #se não tem uma palavra anterior que é igual ou parecida com as tags procuradas, então sai fora:
              if s[fim2:fim2+d+digit_aux][-1].isdigit():
                fim_final = fim2 + 1 + digit_aux
                if fim_final in indices_area: fim_final -=1
              else:
                fim_final = fim2 + digit_aux
                if fim_final in indices_area: fim_final -=1
              #print("FORAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA", s[fim2:fim2+d+1])
              break
          #print(s[inicio_final:fim_final])
        #print("final", s[inicio_final:fim_final])
        frase_final = s[inicio_final:fim_final].strip()
        #print(indices_area, fim_final)
        #print(frase_final)
        list_frases.append((frase_final, len(frase_final)))
  return returnComodosOk(list_frases) 

In [ ]:
"""BOM SE NÃO TEM NÚMERO NO INÍCIO E NO FINAL TAMBÉM. POR EXEMPLO: cod 1541 dorms 2 suites 3 banheiros 2"""
def tagsShow(s, tipo):
  s = re.sub('\(?\d{1,2}\)?\s*\d{4,5}[.-]?\d{3,5}', '', s)
  if tipo == 'comodos':
    s = unidecode(s.lower().replace('(','').replace(')','').replace(':','').replace('\xa0','').replace('de','').replace('com','').replace('demi-', '').replace('demi',''))  
    #s = unidecode(s.lower().replace(':','').replace('\xa0','').replace('de','').replace('com','').replace('demi-', '').replace('demi',''))  
    list_variaveis = ['dorm', 'quarto', 'suit', 'vaga', 'garage', 'banheiro', 'carro']
    vs1 = re.findall('[dormquartosvagasuitegaragembanheiroscarro\s]+\d{1,2}', s) 
    vs2 = re.findall('\d{1,2}[dormquartosvagasuitegaragembanheiroscarro\s]+', s)
    indices_area = indicesArea(s)
    indices_preco = indicesPreco(s)
    indices_area +=indices_preco
    indices_excess = indices_area
  elif tipo == 'areas':
    s = unidecode(s.lower().replace('(','').replace(')','').replace(':','').replace('\xa0','').replace('de','').replace('com','').replace('m²', '').replace('m2', '').replace('m 2','').replace('m ²','').replace('m22','').replace('casa', ''))
    list_variaveis = ['area','priv','tot','terren','constr']
    vs1 = re.findall('[areatotalconstruidprivativterrenoglobal\s]+\d+[,.]?\d+', s) 
    vs2 = re.findall('\d+[,.]?\d+[areatotalconstruidprivativterrenoglobal\s]+', s)
    indices_preco = indicesPreco(s)
    indices_excess = indices_preco
  vs1.extend(vs2)
  #print(vs1)
  list_indices_and_phase = []
  list_indices_iniciais = []
  for v in vs1:
    #print(s.find(v), s.find(v)+len(v), s[s.find(v):s.find(v)+len(v)])
    #print(s.find(v.strip()), s.find(v.strip())+len(v.strip()), s[s.find(v.strip()):s.find(v.strip())+len(v.strip())])
    for lv in list_variaveis:
      if lv in v:
        #print(v,  s.find(v), s.find(v)+len(v))
        v = v.strip()
        indice_v_inicial, indice_v_final = s.find(v), s.find(v)+len(v)
        if indice_v_inicial in indices_excess or indice_v_final-1 in indices_excess:continue
        if indice_v_inicial in indices_excess or indice_v_final-1 in indices_excess:continue
        if indice_v_inicial in list_indices_iniciais:continue
        list_indices_iniciais.append(indice_v_inicial)
        list_indices_and_phase.append((indice_v_inicial, indice_v_final, s[indice_v_inicial:indice_v_final]))
        break
  list_indices_and_phase.append((2000000000,-1,-1))
  #print(s)
  list_indices_and_phase.sort(key=lambda x: x[0])
  #print("LISTA", list_indices_and_phase)
  dorms, quartos, suites, vagas, banheiros, garagem, carros  = 'NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM'
  area, area_priv, area_terreno, area_construida, area_total, area_global, area_util = 'NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM'
  pula = 0
  for l in range(len(list_indices_and_phase)):
    if pula == 1:
      pula = 0
      continue
    frase = ''
    if l == len(list_indices_and_phase)-1 :break#or l == len(list_indices_and_phase)-1:break
    if list_indices_and_phase[l][1] in list(range(list_indices_and_phase[l+1][0], list_indices_and_phase[l+1][1]+1)):
      n = re.findall('\d+', list_indices_and_phase[l][2])[0]
      #print( "IGUAL:", re.findall('\D+', list_indices_and_phase[l][2].strip())[0].strip(), re.findall('\D+', list_indices_and_phase[l+1][2])[0].strip())
      #print(list_indices_and_phase[l][2].split()[-1], list_indices_and_phase[l+1][2].split()[0], list_indices_and_phase[l][2].split()[-1] == list_indices_and_phase[l+1][2].split()[0])
      if list_indices_and_phase[l+1][2].strip()[:len(n)] == n:
        #print("PRIMEIRO", list_indices_and_phase[l][2].strip())
        pula = 1
        frase = list_indices_and_phase[l][2].strip()
      elif re.findall('\D+', list_indices_and_phase[l][2])[0].strip() == re.findall('\D+', list_indices_and_phase[l+1][2])[0].strip():
        frase = list_indices_and_phase[l][2].strip()
        #if tipo == 'areas':pula = 1
        #pula = 1
        #print("FRASE IGUAL", frase)
      #se a palavra final for diferente da palavra inicial a outra frase
      elif list_indices_and_phase[l][2].split()[-1] != list_indices_and_phase[l+1][2].split()[0]:
        frase = list_indices_and_phase[l][2].strip()
        #print("FRASE DIFERENTE", frase)
      
    else:
      #print(list_indices_and_phase[l][2].strip())
      frase = list_indices_and_phase[l][2]

    if frase == '':continue
    #print(frase)
    if tipo == 'comodos':
      if 'dorm' in frase: 
        if re.findall('\d{1,2}\s*dorm|dorm\w*\s*\d{1,2}', frase) == []:
          #print("não pertence")
          continue
        if dorms == 'NENHUM':dorms = re.findall('\d+', frase)[0]
        else: dorms +='&-&'+ re.findall('\d+', frase)[0]
      if 'quarto' in frase: 
        if re.findall('\d{1,2}\s*quarto|quarto\w*\s*\d{1,2}', frase) == []:continue
        if quartos == 'NENHUM':quartos = re.findall('\d+', frase)[0]
        else: quartos +='&-&'+ re.findall('\d+', frase)[0]
      if 'suit' in frase: 
        if re.findall('\d{1,2}\s*suit|suit\w*\s*\d{1,2}', frase) == []:continue
        if suites == 'NENHUM':suites = re.findall('\d+', frase)[0]
        else: suites +='&-&'+ re.findall('\d+', frase)[0]
      if 'vaga' in frase: 
        if re.findall('\d{1,2}\s*vaga|vaga\w*\s*\d{1,2}', frase) == []:continue
        if vagas == 'NENHUM':vagas = re.findall('\d+', frase)[0]
        else: vagas +='&-&'+ re.findall('\d+', frase)[0]
      if 'banheiro' in frase: 
        if re.findall('\d{1,2}\s*banheiro|banheiro\w*\s*\d{1,2}', frase) == []:continue
        if banheiros == 'NENHUM':banheiros = re.findall('\d+', frase)[0]
        else: banheiros +='&-&'+ re.findall('\d+', frase)[0]
      if 'garag' in frase: 
        if re.findall('\d{1,2}\s*garag|garag\w*\s*\d{1,2}', frase) == []:continue
        if garagem == 'NENHUM':garagem = re.findall('\d+', frase)[0]
        else: garagem +='&-&'+ re.findall('\d+', frase)[0]
      if 'carro' in frase: 
        if re.findall('\d{1,2}\s*carro|carro\w*\s*\d{1,2}', frase) == []:continue
        if carros == 'NENHUM':carros = re.findall('\d+', frase)[0]
        else: carros +='&-&'+ re.findall('\d+', frase)[0]
    elif tipo == 'areas':
      if len(re.findall('\d', frase)) >=8: continue #NÚMERO DE TELEFONE
      if 'priv' in frase: 
        if area_priv == 'NENHUM':
          area_priv = re.findall('\d+[,.]?\d+', frase)[0]
        else: area_priv +='&-&'+re.findall('\d+[,.]?\d+', frase)[0]
      if 'terren' in frase: #area_terreno = re.findall('\d+[,.]?\d+', frase)[0]
        if area_terreno == 'NENHUM':
          area_terreno = re.findall('\d+[,.]?\d+', frase)[0]
        else: area_terreno +='&-&'+re.findall('\d+[,.]?\d+', frase)[0]
      if 'constru' in frase: #area_construida = re.findall('\d+[,.]?\d+', frase)[0]
        if area_construida == 'NENHUM':
          area_construida = re.findall('\d+[,.]?\d+', frase)[0]
        else: area_construida +='&-&'+re.findall('\d+[,.]?\d+', frase)[0]
      if 'util' in frase: #area_util = re.findall('\d+[,.]?\d+', frase)[0]
        if area_util == 'NENHUM':
          area_util = re.findall('\d+[,.]?\d+', frase)[0]
        else: area_util +='&-&'+re.findall('\d+[,.]?\d+', frase)[0]
      if 'global' in frase: #area_global = re.findall('\d+[,.]?\d+', list_indices_and_phase[l][2])[0]
        if area_global == 'NENHUM':
          area_global = re.findall('\d+[,.]?\d+', frase)[0]
        else: area_global +='&-&'+re.findall('\d+[,.]?\d+', frase)[0]
      if 'area' in frase and ('priv' not in frase or 'constru' not in frase or 'terren' not in frase or 'la' not in frase): #area = re.findall('\d+[,.]?\d+', frase)[0]
        if area == 'NENHUM':
          area = re.findall('\d+[,.]?\d+', frase)[0]
        else: area +='&-&'+re.findall('\d+[,.]?\d+', frase)[0]
      if 'total' in frase and ('priv' not in frase or 'constru' not in frase or 'terren' not in frase or 'la' not in frase): #area_total = re.findall('\d+[,.]?\d+', frase)[0]
        if area_total == 'NENHUM':
          area_total = re.findall('\d+[,.]?\d+', frase)[0]
        else: area_total +='&-&'+re.findall('\d+[,.]?\d+', frase)[0]

  if tipo == 'comodos':
    return dorms, quartos, suites, vagas, banheiros, garagem, carros  
  elif tipo == 'areas': return area, area_priv, area_terreno, area_construida, area_total, area_global, area_util

In [ ]:
def regexOU(sentence):
  dorms_ou, quartos_ou, suites_ou, banheiros_ou, vagas_ou, garagem_ou, carros_ou, box_ou = 'NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM'
  sentence = re.sub('\s+', ' ', unidecode(sentence.lower()))
  for tag in ['dorm','quarto','suit','banheiro','vaga','garage','carro','box']:
    regex1 = re.findall('\d{1,2}\s*[aou]+\s*\d{1,2}\s*'+tag, sentence)
    regex2 = re.findall('\d{1,2},\s*\d{1,2}\s*[aou]+\s*\d{1,2}\s*'+tag, sentence)
    str_regex, str_regex1, str_regex2 = 'NENHUM', 'NENHUM', 'NENHUM'
    if regex2 != []:
      #print(regex2, df_sentences['Sentence'].iloc[i])
      for v in list(set(regex2)):
        if str_regex == 'NENHUM':str_regex = v
        else: str_regex +='&-&'+v
      #print(str_regex)
    elif regex1 != []:
      for v in list(set(regex1)):
        if str_regex == 'NENHUM':str_regex = v
        else: str_regex +='&-&'+v
      #print(str_regex)
    if str_regex != 'NENHUM':
      if tag == 'dorm':dorms_ou = str_regex.replace(tag, '').strip()
      if tag == 'quarto':quartos_ou = str_regex.replace(tag, '').strip()
      if tag == 'suit':suites_ou = str_regex.replace(tag, '').strip()
      if tag == 'banheiro':banheiros_ou = str_regex.replace(tag, '').strip()
      if tag == 'vaga':vagas_ou = str_regex.replace(tag, '').strip()
      if tag == 'garage':garagem_ou = str_regex.replace(tag, '').strip()
      if tag == 'carro':carros_ou = str_regex.replace(tag, '').strip()
      if tag == 'box':boxs_ou = str_regex.replace(tag, '').strip()
  return dorms_ou, quartos_ou, suites_ou, banheiros_ou, vagas_ou, garagem_ou, carros_ou, box_ou

# Funções Regex

## Pré-processamento nlp6

In [ ]:
def configuraTexto(value):
  digt = re.findall('\d+', value)
  if digt != []:
    digt = digt[0]
    l = value.split(digt)
    for k in range(len(l)):
      if l[k]=='':l[k] = digt
    word_rep = ' '.join(l)
    return word_rep

In [ ]:
def tokenize_sentence(sent):
  list_tk = []
  for token_sent in nltk.word_tokenize(sent):
    list_tk.append(token_sent)
  tokens_sent = [plv for plv in list_tk if plv not in stopwords and plv not in string.punctuation ]
  return ' '.join([str(elemento) for elemento in tokens_sent])

In [ ]:
#EEEEEEEEEEEEEEEEEEEEEEEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE
list_types_names_tags = []
list_type_dorms = []
list_type_vagas = []
list_type_suites = []
list_numbers = ['1','2','3','4','5','6','7','8','9']
#df_sentence_and_tags = pd.read_csv('/content/gdrive/My Drive/COOP/Anotador/AnotadorSpacyO5.csv')
def findWordsAndIndices(sentence_find, word_find, tag, sub_word_padrao, option_ner = '*', sentence_ner=''):
  for i in range(1):
    word_find_sub_not_replace = word_find
    s, inicio, fim = '','', ''
    if option_ner == 1 or option_ner == 2:
      sentence_find = sentence_ner
    if word_find_sub_not_replace in sentence_find:
      s = sentence_find
      if option_ner != 1:
        s = s.replace('ormit ori', 'ormitóri')
        s = s.replace('su ite', 'suíte')
        s = s.replace(word_find_sub_not_replace, sub_word_padrao)
        s = s.replace(sub_word_padrao+'.', sub_word_padrao)
        s = s.replace('dormitoriosór', 'dormitorios')
        s = s.replace('dormitoriosio', 'dormitorios')
        s = s.replace('dormitoriosorios', 'dormitorios')
        s = s.replace('dormitoriositorios', 'dormitorios')
        s = s.replace('dormitoriositorio(s)', 'dormitorios')
        s = s.replace('dormitoriositório(s)', 'dormitorios')
        s = s.replace('dormitoriositórios', 'dormitorios')
        s = s.replace('Vaga(s):', 'vagas')
        s = s.replace('vaga(s):', 'vagas')
        s = s.replace('suítestestes', 'suítes')
        regex = '[0-9]{1}'+word_find_sub_not_replace+'\w*'
        value = re.findall(regex, s)
        if value != []: 
          text = configuraTexto(value[0])
          s = s.replace(value[0], text)
        regex = word_find_sub_not_replace+'\w*'+'[0-9]{1}'
        value = re.findall(regex, s)
        if value != []: 
          text = configuraTexto(value[0])
          s = s.replace(value[0], text)
      word_find = sub_word_padrao
      #02 dormitorios
      if s[s.find(word_find)-1] == ' ' and s[s.find(word_find)-2] != '0' and re.findall('\d',s[s.find(word_find)-2]) != [] and s[s.find(word_find)-3] == '0' and s.find(word_find) != -1:
        for x in range(18):
          try:
            #print(s[s.find(word_find)+x])
            if s[s.find(word_find)+x]==' ' or s[s.find(word_find)+x]=='.':               
              inicio = s.find(word_find)-3
              fim = s.find(word_find)+x
              if option_ner == 1: return inicio, fim
              word_sub = s[inicio:fim].replace(s[inicio+3:fim], sub_word_padrao)
              s = s.replace(s[inicio+3:fim], sub_word_padrao)
              s = s.replace(sub_word_padrao+'s', sub_word_padrao)
              init = s.find(s[s.find(word_sub)])
              init = s.find(word_sub)
              end = init + len(sub_word_padrao) + 3
              #print("2 ------",s[init:end], "2.1--",s[inicio:fim])
              inicio, fim = init, end
              break
          except: break
      #2 dormitórios
      elif s[s.find(word_find_sub_not_replace)-1] == ' ' and s[s.find(word_find_sub_not_replace)-2] != '0' and re.findall('\d',s[s.find(word_find_sub_not_replace)-2]) != [] and s.find(word_find_sub_not_replace) != -1 :
        for x in range(18):
          try: 
            if s[s.find(word_find_sub_not_replace)+x]==' ' or s[s.find(word_find_sub_not_replace)+x]==';' or s[s.find(word_find_sub_not_replace)+x]=='.': 
              #print("opa",s[s.find(word_find_sub_not_replace)-2:s.find(word_find_sub_not_replace)+x])
              inicio = s.find(word_find_sub_not_replace)-2
              fim = s.find(word_find_sub_not_replace)+x
              if option_ner == 1: return inicio, fim
              word_sub = s[inicio:fim].replace(s[inicio+2:fim], sub_word_padrao)
              s = s.replace(s[inicio+2:fim], sub_word_padrao)
              #if s[fim-2:fim+1] == 'ss': s=s.replace(s[fim-2:fim+1], sub_word_padrao)
              s = s.replace(sub_word_padrao+'s', sub_word_padrao)
              init = s.find(word_sub)
              end = init + len(sub_word_padrao) + 2
              #print("2 ------",s[init:end], "2.1--",s[inicio:fim])              
              inicio, fim = init, end              
              break
          except:break
      #dormitórios 2 e dormitórios2
      else:
        space = 0
        for j in range(30):
          if s.find(word_find) != -1:
            try:
              if s[s.find(word_find)+j] == ',':break
              if s[s.find(word_find)+j] == ' ': space +=1
              if space == 2:break
              if s[s.find(word_find)+j] in list_numbers:
                inicio = s.find(word_find)
                fim = s.find(word_find)+j
                if option_ner == 1:return inicio, fim+1
                if s[fim] == '0' or re.findall('\d', s[fim]) == []: fim -=1            
                if s[fim] == ' ': fim -=1
                s = s.replace(sub_word_padrao+'s', sub_word_padrao)
                init = s.find(word_find)
                if s.find(sub_word_padrao+'s') != -1:
                  end = init + len(sub_word_padrao) + 2
                else: end = fim+1
                inicio, fim = init, end
                if s[end] == ' ':end -=1
                break
            except: break
      if option_ner == 1 or option_ner == 2:
        break
    else: 
      if option_ner == 2:return sentence_find
      else: return 0,0
    s = s.replace('dormitoriositórios', 'dormitorios')
  if option_ner == 2: return s
  if option_ner == 1 and inicio != '' and fim != '' and inicio !=-1 and fim != -1:
    return inicio, fim, tag, s
  return 0,0

In [ ]:
def removeTags(text, list_tags_more_spaces = 'null', imob_excec='null', list_regex_excec='null'):

  #scripts = re.compile(r'<script.*?/script>')
  scripts = re.compile(r'<(script).*?</\1>(?s)')
  css = re.compile(r'<style.*?/style>')
  header = re.compile(r'<header.*?/header>')
  footer = re.compile(r'<footer.*?/footer>')
  tags = re.compile(r'<.*?>')
  exc = re.compile(r'\xa0')
  nbsp = re.compile(r'&nbsp;')
  nbsp1 = re.compile(r'&nbsp')
  nbsp2 = re.compile(r'nbsp')
  amp = re.compile(r'&amp;')
  ampliar = re.compile(r'Ampliar')
  direcao = re.compile(r' -->')

  if list_tags_more_spaces != 'null':
    for regex_more_space in list_tags_more_spaces:
      text = re.sub(regex_more_space, '     [SPACE]     ', text)

  if list_regex_excec != 'null':
    for regex_excec in list_regex_excec:
      text = re.sub(regex_excec, '', text)

  text = scripts.sub('', text)
  text = css.sub('', text)
  text = header.sub(' ', text)
  text = tags.sub(' ', text)
  text = exc.sub(' ', text)
  text = nbsp.sub(' ', text)
  text = nbsp1.sub(' ', text)
  text = nbsp2.sub(' ', text)
  text = amp.sub(' ', text)
  text = ampliar.sub('', text)
  text = direcao.sub(' ', text)
  text = footer.sub('', text)
  return text

## Processos textos

In [ ]:
def regexDormsNumberBefore(text, tag):
    list_words = []
    label = "TAGS"
    doc = nlp(text)
    ex = r'\d{1,2}\s*?'+tag+'[\w]*[(s)]*?[-:)(]*?'
    expression = re.compile(ex)
    for match in re.finditer(expression, doc.text):
        already = 0
        start, end = match.span()
        span = doc.char_span(start, end, label=label)
        if span:
          for i in range(0,40):
            if end+i >= len(text):break
            if text[end+i] != ' ':
              if re.findall('\d', text[end+i]) != []:
                already = 1
              break
        if already == 0 and start != -1:
          list_words.append({'palavra':span, 'inicio':start, 'fim':end})
    return list_words

In [ ]:
def regexDormsNumberNext(text, tag):
    list_words = []
    label = "TAGS"
    doc = nlp(text)
    ex = tag+'[\w]*[(s)]*?[-:)(]*?\s*?\d{1,2}'
    already = 0
    expression = re.compile(ex)
    for match in re.finditer(expression, doc.text):
        start, end = match.span()
        span = doc.char_span(start, end, label=label)
        if span:
          #se tiver um número antes, não adiciona
          for i in range(1,40):
            if text[start-i] != ' ':
              if re.findall('\d', text[start-i]) != []:
                already = 1
              break
        if already == 0 and start != -1:
          list_words.append({'palavra':span, 'inicio':start, 'fim':end})
    return list_words

In [ ]:
#list_entities = [(132, 140, 'TIPO'), (237, 277, 'ADDRESS'), (224, 229, 'COD'), (294, 306, 'VALOR'), (311, 323, 'VALOR'), (766, 779, 'VALOR'), (790, 803, 'VALOR'), (828, 840, 'VALOR'), (766, 779, 'VALOR'), (1379, 1392, 'VALOR'), (1379, 1392, 'VALOR'), (1644, 1657, 'VALOR'), (1644, 1657, 'VALOR'), (1924, 1937, 'VALOR'), (2055, 2068, 'VALOR'), (2179, 2192, 'VALOR'), (2349, 2362, 'VALOR'), (1644, 1657, 'VALOR'), (2055, 2068, 'VALOR'), (2786, 2799, 'VALOR'), (3427, 3438, 'VALOR'), (486, 495, 'AREA'), (524, 532, 'AREA'), (1333, 1340, 'AREA'), (1333, 1340, 'AREA'), (1609, 1618, 'AREA'), (1742, 1751, 'AREA'), (1888, 1897, 'AREA'), (2020, 2029, 'AREA'), (2143, 2152, 'AREA'), (2301, 2310, 'AREA'), (2457, 2466, 'AREA'), (2604, 2613, 'AREA'), (2750, 2759, 'AREA'), (2869, 2878, 'AREA')]
def verifyUniquesEntities(list_entities, tuple_tags, modo):
  if modo == 'insert':
    list_entities2 = []
    list_entities2.append(list_entities)
    for tag_word in tuple_tags:
      ai = tag_word[0]
      af = tag_word[1]
      l = [x for x in range(ai, af+1)]
      for j in range(len(list_entities)):
          if list_entities[j][0] not in l and list_entities[j][1] not in l:
            list_entities2.append(tag_word)
    return list_entities2
  #se não tiver outra tag entre o índice de outra tag
  elif modo == 'certify':
    for tag_word in tuple_tags:
      ai = tag_word[0]
      af = tag_word[1]
      l = [x for x in range(ai, af+1)]
      for j in range(len(list_entities)):
          if list_entities[j][0] in l or list_entities[j][1] in l:
            return 0 #já tem uma tag entre os índices
    return 1

In [ ]:
def removeTags(text, list_tags_more_spaces = 'null', list_remove_tags = 'null', imob_excec='null', list_regex_excec='null'):

  #scripts = re.compile(r'<script.*?/script>')
  scripts = re.compile(r'<(script).*?</\1>(?s)')
  css = re.compile(r'<style.*?/style>')
  header = re.compile(r'<header.*?/header>')
  footer = re.compile(r'<footer.*?/footer>')
  tags = re.compile(r'<.*?>')
  exc = re.compile(r'\xa0')
  nbsp = re.compile(r'&nbsp;')
  nbsp1 = re.compile(r'&nbsp')
  nbsp2 = re.compile(r'nbsp')
  amp = re.compile(r'&amp;')
  ampliar = re.compile(r'Ampliar')
  direcao = re.compile(r' -->')

  if list_remove_tags != 'null':
    for r in list_remove_tags:
      classe = r['Class']
      id = r['id']
      tag = r['TAG']
      soup_r = BeautifulSoup(text,"html5")
      text = soup_r
      try:
        if classe != '':
          soup_r.find(class_ = classe).decompose()
        if id != '':
          soup_r.find(id = id).decompose()
        if tag != '':
          soup_r.find(tag).decompose()
      except:
        print("ERROR DECOMPOSE TAG REMOVE")
      text = str(text)

  if list_tags_more_spaces != 'null':
    for regex_more_space in list_tags_more_spaces:
      text = re.sub(regex_more_space, '     [SPACE]     ', text)

  if list_regex_excec != 'null':
    for regex_excec in list_regex_excec:
      text = re.sub(regex_excec, '', text)

  text = scripts.sub('', text)
  text = css.sub('', text)
  text = header.sub(' ', text)
  text = tags.sub(' ', text)
  text = exc.sub(' ', text)
  text = nbsp.sub(' ', text)
  text = nbsp1.sub(' ', text)
  text = nbsp2.sub(' ', text)
  text = amp.sub(' ', text)
  text = ampliar.sub('', text)
  text = direcao.sub(' ', text)
  text = footer.sub('', text)
  return text

In [ ]:
def regexDormsNumberBefore(text, tag):
    list_words = []
    label = "TAGS"
    doc = nlp(text)
    ex = r'\d{1,2}\s*?'+tag+'[\w]*[(s)]*?[-:)(]*?'
    expression = re.compile(ex)
    for match in re.finditer(expression, doc.text):
        already = 0
        start, end = match.span()
        span = doc.char_span(start, end, label=label)
        if span:
          for i in range(0,40):
            if end+i >= len(text):break
            if text[end+i] != ' ':
              if re.findall('\d', text[end+i]) != []:
                already = 1
              break
        if already == 0 and start != -1:
          list_words.append({'palavra':span, 'inicio':start, 'fim':end})
    return list_words

In [ ]:
def regexDormsNumberNext(text, tag):
    list_words = []
    label = "TAGS"
    doc = nlp(text)
    ex = tag+'[\w]*[(s)]*?[-:)(]*?\s*?\d{1,2}'
    already = 0
    expression = re.compile(ex)
    for match in re.finditer(expression, doc.text):
        start, end = match.span()
        span = doc.char_span(start, end, label=label)
        if span:
          #se tiver um número antes, não adiciona
          for i in range(1,40):
            if text[start-i] != ' ':
              if re.findall('\d', text[start-i]) != []:
                already = 1
              break
        if already == 0 and start != -1:
          list_words.append({'palavra':span, 'inicio':start, 'fim':end})
    return list_words

In [ ]:
#list_entities = [(132, 140, 'TIPO'), (237, 277, 'ADDRESS'), (224, 229, 'COD'), (294, 306, 'VALOR'), (311, 323, 'VALOR'), (766, 779, 'VALOR'), (790, 803, 'VALOR'), (828, 840, 'VALOR'), (766, 779, 'VALOR'), (1379, 1392, 'VALOR'), (1379, 1392, 'VALOR'), (1644, 1657, 'VALOR'), (1644, 1657, 'VALOR'), (1924, 1937, 'VALOR'), (2055, 2068, 'VALOR'), (2179, 2192, 'VALOR'), (2349, 2362, 'VALOR'), (1644, 1657, 'VALOR'), (2055, 2068, 'VALOR'), (2786, 2799, 'VALOR'), (3427, 3438, 'VALOR'), (486, 495, 'AREA'), (524, 532, 'AREA'), (1333, 1340, 'AREA'), (1333, 1340, 'AREA'), (1609, 1618, 'AREA'), (1742, 1751, 'AREA'), (1888, 1897, 'AREA'), (2020, 2029, 'AREA'), (2143, 2152, 'AREA'), (2301, 2310, 'AREA'), (2457, 2466, 'AREA'), (2604, 2613, 'AREA'), (2750, 2759, 'AREA'), (2869, 2878, 'AREA')]
def verifyUniquesEntities(list_entities, tuple_tags, modo):
  if modo == 'insert':
    list_entities2 = []
    list_entities2.append(list_entities)
    for tag_word in tuple_tags:
      ai = tag_word[0]
      af = tag_word[1]
      l = [x for x in range(ai, af+1)]
      for j in range(len(list_entities)):
          if list_entities[j][0] not in l and list_entities[j][1] not in l:
            list_entities2.append(tag_word)
    return list_entities2
  #se não tiver outra tag entre o índice de outra tag
  elif modo == 'certify':
    for tag_word in tuple_tags:
      ai = tag_word[0]
      af = tag_word[1]
      l = [x for x in range(ai, af+1)]
      for j in range(len(list_entities)):
          if list_entities[j][0] in l or list_entities[j][1] in l:
            return 0 #já tem uma tag entre os índices
    return 1

## Bairros e Imagens

In [ ]:
#ESSE
def getBairros(page):
  doc = nlp(page.lower())
  expression = re.compile('bairro')
  len_bairro = len('bairro')
  caracteres_especiais = [',', '.', '-', '|','(',')']
  list_bairros = []
  for match in re.finditer(expression, doc.text):
    start, end = match.span()
    span = doc.char_span(start, end, label='BAIRRO')
    if span:
      cont_letras, cont_space = 0,0
      for i in range(0,60):
        try:
          if page[end+len_bairro+i] == ' ' and cont_letras > 3:
            cont_space +=1
          elif page[end+len_bairro+i] != ' ':
            cont_space = 0
            cont_letras +=1
          else: cont_space = 0
        except:break
        if cont_space >= 2 or page[end+len_bairro+i] in caracteres_especiais:
          #print(page[start+len_bairro: end+len_bairro+i])
          list_bairros.append(page[start+len_bairro: end+len_bairro+i].strip())
          break
  return list_bairros

In [ ]:
def getImages(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao = []):
  imgs = soup.find_all(tag_img)
  dict_imgs = {}
  for url_check in urls_checks:
    for attr_check in attrs_check:
      list_imgs = []
      for img in imgs:
        try:
          url = img.get(attr_check)
          if url_check in url.lower() and url_check != '':
            list_imgs.append(url)
          elif url_check in url.lower():
            list_imgs.append(url)
        except:pass
      list_imgs = list(set(list_imgs))
      list_imgs_return = []
      for ig in list_imgs:
        ops = 0
        for check in check_excessao:
          if check in ig:
            ops = 1
        if ops == 0:
          list_imgs_return.append(ig)
        dict_imgs[attr_check] = list_imgs_return
  return dict_imgs

In [ ]:
def getImagesAriotti(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao = []):
  imgs = soup.find_all(tag_img)
  dict_imgs = {}
  list_imgs_return = []
  for url_check in urls_checks:
    for attr_check in attrs_check:
      list_imgs = []
      for img in imgs:
        try:
          url = img.get(attr_check)
          #print(url)
          if url_check in url.lower() and url_check != '':
            list_imgs.append(url)
          elif url_check in url.lower():
            list_imgs.append(url)
        except:pass
      list_imgs = list(set(list_imgs))
      for ig in list_imgs:
        ops = 0
        for check in check_excessao:
          if check in ig:
            ops = 1
        if ops == 0:
          list_imgs_return.append(ig)
      dict_imgs[attr_check] = list(set(list_imgs_return))
  return dict_imgs

#Run 2

In [ ]:
def regexSpacy1(page):
  page = page.strip()

  human_verify = False

  suites_regex, dorms_regex, vagas_regex, banheiros_regex = [], [], [], []
  for name_tag in ['quarto', 'dorm', 'vaga', 'suít', 'suit', 'banheiro']:
    tag_insert = ''
    l = regexDormsNumberNext(page.lower(), name_tag)
    if name_tag == 'quarto' or name_tag == 'dorm': tag_insert = 'DORMS'
    if name_tag == 'vaga': tag_insert = 'VAGAS'
    if name_tag == 'suít' or name_tag == 'suit': tag_insert = 'SUITES'
    if name_tag == 'banheiro': tag_insert = 'BANHEIRO'
    for j in range(len(l)):
      cert = verifyUniquesEntities([], [(l[j]['inicio'], l[j]['fim'], tag_insert)], modo='certify')
      if cert == 1:
        if tag_insert == 'SUITES': suites_regex.append(page[l[j]['inicio']:l[j]['fim']])
        if tag_insert == 'DORMS': dorms_regex.append(page[l[j]['inicio']:l[j]['fim']])
        if tag_insert == 'VAGAS': vagas_regex.append(page[l[j]['inicio']:l[j]['fim']])
        if tag_insert == 'BANHEIRO': banheiros_regex.append(page[l[j]['inicio']:l[j]['fim']])

  for name_tag in ['quarto', 'dorm', 'vaga', 'suít', 'suit', 'banheiro']:
    tag_insert = ''
    l = regexDormsNumberBefore(page.lower(), name_tag)
    if name_tag == 'quarto' or name_tag == 'dorm': tag_insert = 'DORMS'
    if name_tag == 'vaga': tag_insert = 'VAGAS'
    if name_tag == 'suít' or name_tag == 'suit': tag_insert = 'SUITES'
    if name_tag == 'banheiro': tag_insert = 'BANHEIRO'
    for j in range(len(l)):
      cert = verifyUniquesEntities([], [(l[j]['inicio'], l[j]['fim'], tag_insert)], modo='certify')
      if cert == 1:
        if tag_insert == 'SUITES': suites_regex.append(page[l[j]['inicio']:l[j]['fim']])
        if tag_insert == 'DORMS': dorms_regex.append(page[l[j]['inicio']:l[j]['fim']])
        if tag_insert == 'VAGAS': vagas_regex.append(page[l[j]['inicio']:l[j]['fim']])
        if tag_insert == 'BANHEIRO': banheiros_regex.append(page[l[j]['inicio']:l[j]['fim']])

  matcher = Matcher(nlp.vocab)
  pattern_dorms = [ {"LIKE_NUM": True}, {"TEXT":{"REGEX":"\s?dorm"}}]
  pattern_suit = [ {"LIKE_NUM": True}, {"TEXT":{"REGEX":"\s?suít"}} ]
  pattern_suit2 = [ {"LIKE_NUM": True}, {"TEXT":{"REGEX":"\s?suit"}} ]
  pattern_banheiro = [ {"LIKE_NUM": True}, {"TEXT":{"REGEX":"\s?banheiro"}} ]
  pattern_vagas = [ {"LIKE_NUM": True}, {"TEXT":{"REGEX":"\s?vagas"}} ]
  pattern_vagas_garag = [ {"LIKE_NUM": True}, {"TEXT":{"REGEX":"\s?garagem"}} ]

  matcher.add("Dorms", [pattern_dorms])
  matcher.add("Suites", [pattern_suit])
  matcher.add("Suites", [pattern_suit2])
  matcher.add("Banheiro", [pattern_banheiro])
  matcher.add("Vagas", [pattern_vagas])
  matcher.add("Vagas", [pattern_vagas_garag])

  doc = nlp(page.lower())
  matches = matcher(doc)
  dorms_spacy_rg,suites_spacy_rg,banheiro_spacy_rg,vagas_spacy_rg = [],[],[],[]
  for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  
    span = doc[start:end] 
    if string_id == 'Dorms': dorms_spacy_rg.append(span.text)
    if string_id == 'Suites': suites_spacy_rg.append(span.text)
    if string_id == 'Banheiro': banheiro_spacy_rg.append(span.text)
    if string_id == 'Vagas': vagas_spacy_rg.append(span.text)

  return suites_regex, dorms_regex, vagas_regex, banheiros_regex, dorms_spacy_rg,suites_spacy_rg,banheiro_spacy_rg,vagas_spacy_rg

In [ ]:
def coordenadasRegex(soup_regex, imob):
  address_regex_iframe = ''
  list_coords, list_lngs, list_lats = [], [], []
  try:
    r = requests.get(soup_regex.find('iframe').get('src'), headers={'User-Agent': 'Mozilla/5.0'})
    #list_coords = list(set(re.findall('-\d{1,3}[.]\d+', r.text)))
    list_lngs.append(list(set(re.findall('-52[.]\d+', r.text))))
    list_lats.append(list(set(re.findall('-28[.]\d+', r.text))))
    text = r.text
    end = text.find('Brazil')+len('Brazil')
    for i in range(end,1,-1):
      if text[i-1] == '"':
        address_regex_iframe = text[i:end]
        break
  except:pass

  lat_maior, lng_maior = 'null','null'

  list_lngs.append(list(set(re.findall('-52[.]\d+', res.text))))
  list_lats.append(list(set(re.findall('-28[.]\d+', res.text))))

  tam = 0
  for lng in list_lngs:
    if len(lng) > tam:
      tam = len(lng)
      lng_maior = lng[0]

  tam = 0
  for lat in list_lats:
    if len(lat) > tam:
      tam = len(lat)
      lat_maior = lat[0]
  if lat_maior == 'null' or lng_maior == 'null':
    list_coords = []
  else:list_coords = [lat_maior, lng_maior]
  if list_coords == ['-28.2529169', '-52.4044902'] and imob == 'R. Classic': list_coords = []

  return list_coords, address_regex_iframe

In [ ]:
def findWordPerIndice(desc):
    desc_ner = desc.lower()

    suites1, suites2, dorms, vagas, quartos, banheiros_r = [],[],[],[],[],[]
    init, fim = findWordsAndIndices(desc_ner, 'sui', 'SUITES', sub_word_padrao='suí', option_ner = 1, sentence_ner=desc_ner)
    if desc_ner[init:fim] == '':suites1 = []
    else:suites1.append(desc_ner[init:fim])

    init, fim = findWordsAndIndices(desc_ner, 'sui', 'SUITES', sub_word_padrao='suí', option_ner = 1, sentence_ner=desc_ner)
    #suites2 = desc_ner[init:fim]
    if desc_ner[init:fim] == '':suites2 = []
    else:suites2.append(desc_ner[init:fim])

    init, fim = findWordsAndIndices(desc_ner, 'dorm', 'DORMS', sub_word_padrao='dorm', option_ner = 1, sentence_ner=desc_ner)
    #dorms = desc_ner[init:fim]
    if desc_ner[init:fim] == '':dorms = []
    else:dorms.append(desc_ner[init:fim])

    init, fim = findWordsAndIndices(desc_ner, 'vaga', 'VAGAS', sub_word_padrao='vaga', option_ner = 1, sentence_ner=desc_ner)
    #vagas = desc_ner[init:fim]
    if desc_ner[init:fim] == '':vagas = []
    else:vagas.append(desc_ner[init:fim])
  
    init, fim = findWordsAndIndices(desc_ner, 'quarto', 'DORMS', sub_word_padrao='dorm', option_ner = 1, sentence_ner=desc_ner)
    #quartos = desc_ner[init:fim]
    if desc_ner[init:fim] == '':quartos = []
    else:quartos.append(desc_ner[init:fim])
    
    init, fim = findWordsAndIndices(desc_ner, 'banheiro', 'BANHEIROS', sub_word_padrao='banheiro', option_ner = 1, sentence_ner=desc_ner)
    #banheiros_r = desc_ner[init:fim]
    if desc_ner[init:fim] == '':banheiros_r = []
    else:banheiros_r.append(desc_ner[init:fim])

    area_m = re.findall('\d+[,.]?\d+\s?m²', desc_ner)
    area_ha = re.findall('\d+[,.]?\d+\s?ha', desc_ner)
    if area_ha == []:area_ha = re.findall('\d+[,.]?\d+\s?hect', desc_ner)
    
    desc_ner = desc_ner.replace('_', ' ')
    if findWordsAndIndices(desc_ner, 'sui', 'SUITES', sub_word_padrao='suítes', option_ner = 2, sentence_ner=desc_ner) != (0, 0): desc_ner = findWordsAndIndices(desc_ner, 'sui', 'SUITES', sub_word_padrao='suítes', option_ner = 2, sentence_ner=desc_ner)
    if findWordsAndIndices(desc_ner, 'suí', 'SUITES', sub_word_padrao='suítes', option_ner = 2, sentence_ner=desc_ner) != (0, 0): desc_ner = findWordsAndIndices(desc_ner, 'sui', 'SUITES', sub_word_padrao='suítes', option_ner = 2, sentence_ner=desc_ner)
    if findWordsAndIndices(desc_ner, 'vaga', 'VAGAS', sub_word_padrao='vagas', option_ner = 2, sentence_ner=desc_ner) != (0, 0): desc_ner = findWordsAndIndices(desc_ner, 'vaga', 'VAGAS', sub_word_padrao='vagas', option_ner = 2, sentence_ner=desc_ner)
    if findWordsAndIndices(desc_ner, 'dorm', 'DORMS', sub_word_padrao='dormitorios', option_ner = 2, sentence_ner=desc_ner) != (0, 0): desc_ner = findWordsAndIndices(desc_ner, 'dorm', 'DORMS', sub_word_padrao='dormitorios', option_ner = 2, sentence_ner=desc_ner)
    if findWordsAndIndices(desc_ner, 'banheiro', 'BANHEIROS', sub_word_padrao='banheiros', option_ner = 2, sentence_ner=desc_ner) != (0,0): desc_ner = findWordsAndIndices(desc_ner, 'banheiro', 'BANHEIROS', sub_word_padrao='banheiros', option_ner = 2, sentence_ner=desc_ner)

    return suites1, suites2, dorms, vagas, quartos, banheiros_r

# Main

In [ ]:
res = requests.get('https://www.benvegnuimoveis.com.br/imovel/locacao-anual-casas-04-dormitorio-em-passo-fundo/casa-4-dorm-(1-suite)--vila-luiza/14405',
    headers= {'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'})

In [ ]:
page_bruto = res.text
page = re.sub('\s+', ' ', page_bruto)
page = cleaner.clean_html(page)
soup = BeautifulSoup(res.text)

In [ ]:
page.find('cookies para personal')
page[50100:50600]

' background: #2BA7B2; } .fecharIframe{ background-color: #2BA7B2 !important; }</style><div class="modal fade modalCookies" id="modal-cookies" tabindex="-1"><div class="modal-dialog modal-lg"><div class="modal-content"></div></div></div><div class="avisoCookiesFixo cookiesmodal cookiesBottom" id="cookiesmodal"><div class="container"><div class="row"><div class="pull-left txtCookies"><p>Usamos cookies para personalizar e melhorar a sua experiência. Ao navegar neste site, você concorda com a nossa '

In [ ]:
sentence = removeTags(page, list_tags_more_spaces=['</li>'], list_remove_tags = [({'Class':'master-slider', 'id':'', 'TAG':''})], imob_excec='Benvegnú', list_regex_excec=['<div id="footer.*','<div class="modal fade modalCookies.*','id="modal-cookies.*','<div class="avisoCookiesFixo.*', '<div class="master-slider ms-skin-light-3 ms-wk.*<div class="clearfix">', '<div class="imovel__detalhe__simulador".*', '<div id="similares.*', '<section class="imovel__similar produto".*'])
page = sentence.lower()
list_imgs = getImages(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','corretor', 'logo', 'layout'])
page

ERROR DECOMPOSE TAG REMOVE


'    casa 4 dorm (1 suíte) - vila luiza | vila luiza | passo fundo                         (54) 99688.6198                  alugar imoveis    »     casas passo fundo    »     casa 4 dormitorios        casa 4 dorm (1 suíte) - vila luiza      14    cód. l14405        4 dorm. sendo 1 suíte        rua coronel gervasio annes , vila luiza - passo fundo / rs             r$1.120.000,00                                      vaga(s):   6       [space]        área do terreno:  690.00 m²      [space]        privativa casa:  426.00 m²      [space]        mobília:  sim      [space]         área total do terreno  690.00m²  :  11.50m  de largura x  60.00m  de comprimento.             características acabamentos           acabamento em gesso     [space]      laje     [space]      piso cerâmico     [space]      piso de porcelanato     [space]      rebaixamento em gesso     [space]               características da casa           alarme     [space]      alarme monitorado     [space]      ar condicionado   

In [ ]:
df_regex_extract = pd.DataFrame(columns=['Dorms Show','Quartos Show','Suites Show','Vagas Show','Banheiros Show','Garagem Show','Carros Show',
    'Area Show','Area Priv Show','Area Terreno Show','Area Global Show','Area Útil Show','Area Construída Show','Area Total Show',
    'dorms REGEX sentence','only dorms REGEX','only dorms REGEX 2','dorms unique REGEX','only dorms REGEX 3',
    'quartos REGEX sentence','only quartos REGEX','only quartos REGEX 2','only quartos REGEX 3','quartos unique REGEX',
    'vagas REGEX sentence', 'carro REGEX sentence', 'garagem REGEX sentence','only vagas REGEX','only vagas REGEX 2',
    'only vagas REGEX 3','only garagem REGEX','only garagem REGEX 2','only garagem REGEX 3','only carros REGEX 2',
    'only carros REGEX 3','vagas unique REGEX','garagem unique REGEX','carros unique REGEX',
    'suites REGEX sentence','only suites REGEX','only suites REGEX 2','only suites REGEX 3','suites unique REGEX',
    'banheiros REGEX sentence','only banheiros REGEX','only banheiros REGEX 2','only banheiros REGEX 3','banheiros unique REGEX',
    'Dorms OU','Quartos OU','Suites OU','Banheiros OU','Vagas OU','Garagem OU','Carros OU','Box OU',
    'only area REGEX', 'only area priv REGEX', 'only area terreno REGEX', 'area REGEX',
    'banheiros NER', 'suites NER', 'area NER', 'vagas NER', 'quartos NER',
    'dorms NER', 'preço NER', 'dorms REGEX', 'dorms REGEX correct','banheiros REGEX', 'banheiros REGEX correct', 'suites REGEX' , 'suites REGEX correct','vagas REGEX', 
    'vagas REGEX correct', 'quartos REGEX', 'quartos REGEX correct',
    'area REGEX correct', 'area priv REGEX correct', 'preco REGEX correct', 'Sentence', 'Preço All', 'preco REGEX','Imobiliária','Tipo', 'Situacao', 'URL', 'Endereco Soup', 'Pagina'])

In [ ]:
def headerImmobile(dominio_imob_def):
  return {
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
      'Accept-Encoding': 'gzip, deflate',
      'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
      'Connection': 'keep-alive',
      'Host': f'{dominio_imob_def}',
      'Origin': f'{dominio_imob_def}',
      'Referer': f'https://{dominio_imob_def}',
      'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
      'sec-ch-ua-mobile': '?0',
      'sec-ch-ua-platform': '"Windows"',
      'Sec-Fetch-Dest': 'document',
      'Sec-Fetch-Mode': 'navigate',
      'Sec-Fetch-Site': 'same-origin',
      'Sec-Fetch-User': '?1',
      'Upgrade-Insecure-Requests': '1',
      'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36',
  }

In [ ]:
list_ner = []

In [ ]:
df_urls_already_list = []

In [ ]:
#df_urls_extracao_parte_2_ok = pd.DataFrame(columns=['URL', 'Situacao', 'Parte'])
#df_urls_extracao_parte_2_ok.to_csv('/content/gdrive/My Drive/NER/DataFrames/URLS/urls_config-26-04.csv', sep='\t')

In [ ]:
name_urls_extract_todos = '/content/gdrive/My Drive/NER/DataFrames/URLS/urls_extract_TODOS-26-04.csv'
name_urls_config = '/content/gdrive/My Drive/NER/DataFrames/URLS/urls_config-26-04.csv'
name_csv = 'urls_extract_TODOS-26-04.csv'

In [ ]:
while True:
  try:
    df_urls = pd.read_csv(name_urls_extract_todos, sep='\t')
    df_urls.drop_duplicates('URL', inplace = True)
    break
  except:
    print("Dataframe das urls ainda não existe")
    time.sleep(30)
    continue

In [ ]:
#arquivo = diretorio/name_csv
#timestamp_att_df = arquivo.stat().st_mtime

In [ ]:
arquivo_urls_config = diretorio/name_urls_config
timestamp_att_end = arquivo_urls_config.stat().st_mtime

In [ ]:
timestamp_att_end

1651327902.0

In [ ]:
gerar_error_next_except = 'opa'

In [ ]:
name_save_file_csv_and_json = '/content/gdrive/My Drive/NER/DataFrames/Extracao/teste-26-04-parte-1'
name_save_file_csv_and_json = '/content/gdrive/My Drive/NER/DataFrames/Extracao/teste-26-04-parte-1-2'
name_save_file_csv_and_json = '/content/gdrive/My Drive/NER/DataFrames/Extracao/teste-26-04-parte-1-3'
name_save_file_csv_and_json = '/content/gdrive/My Drive/NER/DataFrames/Extracao/teste-26-04-parte-1-4'

In [ ]:
df_regex_extract = pd.read_csv(f'{name_save_file_csv_and_json}.csv', sep='\t')
list_ner = open(f'{name_save_file_csv_and_json}.json')
list_ner = json.load(list_ner)

In [ ]:
def checkTimesIsOpenFile(arquivo_urls_config, timestamp_att_df_start):
  timestamp_att_df_now = arquivo_urls_config.stat().st_mtime #ultima modificação no arquivo
  seconds_difference_is_chenge_file = (datetime.fromtimestamp(timestamp_att_df_now) - datetime.fromtimestamp(timestamp_att_df_start)).seconds

  timestamp_att_df_now_is_open = arquivo_urls_config.stat().st_atime #última vez aberto o arquivo
  seconds_difference_is_open_file = (datetime.fromtimestamp(timestamp_att_df_now_is_open) - datetime.fromtimestamp(timestamp_att_df_start)).seconds

  print("TEMPO DESDE A ÚLTIMA ABERTURA", seconds_difference_is_open_file, seconds_difference_is_chenge_file)
  while seconds_difference_is_open_file < 20 or seconds_difference_is_chenge_file < 20:
    print("loading seconds urls_config")
    if seconds_difference_is_chenge_file < 20:
      time.sleep(20)
      timestamp_att_df_now = arquivo_urls_config.stat().st_mtime
      seconds_difference_is_chenge_file = (datetime.fromtimestamp(timestamp_att_df_now) - datetime.fromtimestamp(timestamp_att_df_start)).seconds
    else:
      time.sleep(20)
      timestamp_att_df_now_is_open = arquivo_urls_config.stat().st_atime
      seconds_difference_is_open_file = (datetime.fromtimestamp(timestamp_att_df_now_is_open) - datetime.fromtimestamp(timestamp_att_df_start)).seconds
  return timestamp_att_df_now

In [ ]:
def checkTimesIsOpenFile(arquivo_urls_config):
  timestamp_att_df_start = time.time()
  timestamp_att_df_now = arquivo_urls_config.stat().st_mtime #ultima modificação no arquivo
  seconds_difference_is_change_file = (datetime.fromtimestamp(timestamp_att_df_now) - datetime.fromtimestamp(timestamp_att_df_start)).seconds
  
  print("TEMPO DESDE O ÚLTIMO", seconds_difference_is_change_file)
  while seconds_difference_is_change_file < 20:
    print("loading seconds urls_config")
    if seconds_difference_is_change_file < 20:
      time.sleep(20)
      timestamp_att_df_now = arquivo_urls_config.stat().st_mtime
      seconds_difference_is_change_file = (datetime.fromtimestamp(timestamp_att_df_now) - datetime.fromtimestamp(timestamp_att_df_start)).seconds
  return timestamp_att_df_now

In [ ]:
from datetime import datetime
def checkTimesIsOpenFile(arquivo_urls_config):
  timestamp_att_df_start = time.time()
  timestamp_att_df_now = arquivo_urls_config.stat().st_mtime #ultima modificação no arquivo
  seconds_difference_is_change_file = timestamp_att_df_start - timestamp_att_df_now
  
  print("TEMPO DESDE O ÚLTIMO", seconds_difference_is_change_file)
  while seconds_difference_is_change_file < 30:
    print("loading seconds urls_config")
    if seconds_difference_is_change_file < 30:
      time.sleep(30)
      timestamp_att_df_start = time.time()
      timestamp_att_df_now = arquivo_urls_config.stat().st_mtime
      seconds_difference_is_change_file = timestamp_att_df_start - timestamp_att_df_now
  return timestamp_att_df_now

In [ ]:
df_regex_extract.shape, len(list_ner)

((0, 89), 0)

In [ ]:
urls_config = pd.read_csv(name_urls_config, sep='\t')

In [ ]:
df_urls = pd.read_csv(name_urls_extract_todos, sep='\t')
list_urls_config = urls_config[(urls_config['Situacao'] == 'OK') | (urls_config['Situacao'] == 'Em extracao')]['URL'].to_list()
#df_urls = df_urls.loc[[x not in list_urls_config for x in df_urls['URL'].tolist()]]
#rows = np.random.choice(df_urls.index.values, 50 if df_urls.shape[0] >= 50 else df_urls.shape[0])
#df_urls = df_urls.loc[rows]
len(list_urls_config), df_urls.shape

(20543, (21109, 7))

In [ ]:
len(urls_config[(urls_config['Situacao'] == 'OK') | (urls_config['Situacao'] == 'Em extracao')]['URL'].to_list())
[x not in list_urls_config for x in df_urls['URL'].tolist()]

In [ ]:
df_urls.duplicated(subset=['URL']).sum()

In [ ]:
urls_config['Situacao'].value_counts()

In [ ]:
import requests

res = requests.get('https://arnelimoveis.com.br/imovel/7127/casa-3-quartos-santa-maria-ii-passo-fundo/')

In [ ]:
page_bruto = res.text

page = re.sub('\s+', ' ', page_bruto)
page = cleaner.clean_html(page)
soup = BeautifulSoup(res.text)
sentence = removeTags(page, list_tags_more_spaces=['</small>'], imob_excec='Arnel', list_regex_excec=['<h2>Semelhantes.*', '<section class="pgl-featured pgl-properties card-semelhantes.*"', 
                                                                                                            '<div class="formEnvia.*', 'card-semelhantes.*' ])
page = sentence.lower()

In [ ]:
page

'               casa com 3 quartos, santa maria ii, passo fundo – r$ 430.000,00 – cod. 7127 – arnel imóveis.                                                                                                                                                                                                                                                                                                                                                                                         cod 7127     [space]          casa de 03 dormitórios no bairro santa maria ii para comprar                                  compartilhe                                                             home     busca imóveis    casa de 03 dormitórios no bairro santa maria ii para comprar               casa em alvenaria, disponibilidade para guardar um segundo carro, suíte master no segundo piso.espaços amplos, bem arejados ótima posição solar.               santa maria ii -    passo fundo/   rs             r$ 430.00

In [ ]:
cont = 1
cont_imobs_extraction_now = 1

while True:
  print("TEMPO 60s")
  time.sleep(5)
  #arquivo = diretorio/name_csv
  #tempo2 = arquivo.stat().st_mtime
  #if tempo2 > timestamp_att_df:
  #  timestamp_att_df = tempo2
  #else:
  #  print("TEMPO NÃO VÁLIDO DE ATUAZLIAÇÃO DO DATAFRAME DE TODAS AS URLS")
  #  continue

  df_urls = pd.read_csv(name_urls_extract_todos, sep='\t')
  if df_urls.shape[0] == 0:
    time.sleep(30)
    continue

  df_urls = df_urls.loc[:, ~df_urls.columns.str.contains('^Unnamed')]

  if cont_imobs_extraction_now != 1:
    timestamp_att_end = checkTimesIsOpenFile(arquivo_urls_config)
  
  urls_config = pd.read_csv(name_urls_config, sep='\t')
  urls_config = urls_config.append({'URL':'aberto/grava', 'Situacao': 'aberto/grava', 'Parte':0}, ignore_index = True)

  urls_config = urls_config.loc[:, ~urls_config.columns.str.contains('^Unnamed')]
  print(df_urls.shape, "- URLS CONFIG:", urls_config.shape[0])

  if urls_config[urls_config['Situacao'] == 'OK'].shape[0] == df_urls.shape[0]:break

  if urls_config.shape[0] > 0:
    list_urls_config = urls_config[(urls_config['Situacao'] == 'OK') | (urls_config['Situacao'] == 'Em extracao')]['URL'].to_list()
    df_urls = df_urls.loc[[x not in list_urls_config for x in df_urls['URL'].tolist()]]
    rows = np.random.choice(df_urls.index.values, 50 if df_urls.shape[0] >= 50 else df_urls.shape[0])
    df_urls = df_urls.loc[rows]
  else:
    rows = np.random.choice(df_urls.index.values, 50 if df_urls.shape[0] >= 50 else df_urls.shape[0])
    df_urls = df_urls.loc[rows]
        
  print("Selecionou", df_urls['URL'].shape)
  timestamp_att_end = checkTimesIsOpenFile(arquivo_urls_config)
  contilk = 0
  for ilk in range(df_urls.shape[0]):
    #urls_config = urls_config.append({'URL':df_urls['URL'].iloc[ilk], 'Situacao': 'OK'}, ignore_index = True)
    contilk +=1
    urls_config = urls_config.append({'URL':df_urls['URL'].iloc[ilk], 'Situacao': 'Em extracao', 'Parte':0}, ignore_index = True)
  urls_config.to_csv(name_urls_config, sep='\t')
  print("ADICIONOU", contilk)

  list_urls_novos = []
  indice_df = -1
  list_urls_already_in_df = df_regex_extract['URL'].to_list()
  for imob, url, tipo, situacao in zip(df_urls['Imobiliária'], df_urls['URL'], df_urls['Tipo'], df_urls['Situacao']):
    cont +=1
    if url in list_urls_already_in_df:continue
    print(cont_imobs_extraction_now, imob, url, tipo, situacao)
    indice_df +=1
    #if indice_df <= df_regex_extract_aux.shape[0]:continue
    dominio_imob = df_urls['Dominio'].iloc[indice_df]

    if dominio_imob == '' or dominio_imob == None or type(dominio_imob) == float:
      header_immobile = {'User-Agent': 'Mozilla/5.0'}
    else:
      header_immobile = headerImmobile(dominio_imob_def = dominio_imob)
    header_immobile = {'User-Agent': 'Mozilla/5.0'}
    imob = imob.strip()  

    try:
      res = requests.get(url, headers = header_immobile) #headers={'User-Agent': 'Mozilla/5.0'})
      res.encoding = 'UTF-8'
    except:
      print("ERRO URL")
      try:
        df_urls['Situacao'].iloc[df_urls[df_urls['URL'] == url].index.to_list()] = 'ERROR URL'
        print("deu certo ERROR URL")
      except:print("ERROR SITUACAO URL")
      continue
    page_bruto = res.text

    page = re.sub('\s+', ' ', page_bruto)
    try:
      page = cleaner.clean_html(page)
    except:
      print("ERROR CLEANER")
      continue
    soup = BeautifulSoup(res.text)

    #if '404' in res.text or 'página não encontrada' in res.text.lower():
      #print("ERROR 404", url)
      #continue
    address_soup, sentence = 'NENHUM', 'NENHUM'
    if imob == 'Bortolini':
      sentence = removeTags(page, list_tags_more_spaces=['<div class="item">'],  imob_excec='Bortolini', list_regex_excec=['<div class="imovel__detalhe__simulador".*', '<section class="imovel__similar produto".*'])
      page = sentence.lower()
      list_imgs = getImages(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','corretor', 'logo', 'layout', 'uploads'])
      try:
        address_soup = soup.find(class_='imovel__detalhe__header').find(class_='local').text.strip()
      except:pass
    elif imob == 'Associadas' or imob == 'Ativa':
      sentence = removeTags(page, list_tags_more_spaces=['</li>'], list_remove_tags = [({'Class':'', 'id':'sigaGaleria', 'TAG':''})], imob_excec='Associadas', list_regex_excec=['<p class="rodape.*', '<h3 class="similares no-border padding-bottom-15 padding-top-10".*', '<div id="similares.*', '<div class="col-lg-4 col-md-5 stickyForm.*class="fimRolagemContato">', '<div class="modal fade modalCookies.*','id="modal-cookies.*','<div class="avisoCookiesFixo.*'])
      page = sentence.lower()
      list_imgs = getImages(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','corretor', 'logo', 'layout'])
      try:
        address_soup = soup.find(class_='sp-address-property').text.strip()
      except:pass
    elif imob == 'Benvegnú Imóveis' or imob == 'Benvegnu' or imob == 'Imóveis endres' or imob == 'Imobiliárias Coligadas' or imob == 'Bellotti' or imob == 'Ativa' or imob == 'Ceolin Imóveis' or imob == 'Bins Imobiliária' or imob == 'Medina' or imob == 'Bérgamo Imobiliária' or imob == 'Espendlor Imóveis' or imob == 'Casa Tua Transações Imobiliárias':
      sentence = removeTags(page, list_tags_more_spaces=['</li>'], list_remove_tags = [({'Class':'master-slider', 'id':'', 'TAG':''})], imob_excec='Benvegnú', list_regex_excec=['<div class="master-slider ms-skin-light-3 ms-wk.*<div class="clearfix">', '<div id="footer.*', '<div class="imovel__detalhe__simulador".*', '<div id="similares.*', '<section class="imovel__similar produto".*', '<div class="modal fade modalCookies.*','id="modal-cookies.*','<div class="avisoCookiesFixo.*'])
      page = sentence.lower()
      list_imgs = getImages(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','corretor', 'logo', 'layout'])
      try:
        address_soup = soup.find(class_='sp-address-property').text.strip()
      except:pass
    elif imob == 'Finardi Carrão Imóveis' or imob == 'Finardi Carrão' or imob == 'Aluga Administradora de Bens LTDA' or imob == 'Fragomeni Imóveis' or imob == 'Gilberto Pedott Corretor de Imóveis' or imob == 'Cau Imóveis' or imob == 'Bertolini e Schneider Imóveis' or imob == 'Graffer Negócios Imobiliários' or imob == 'Área Nobre Imóveis':
      #sentence = removeTags(page, list_tags_more_spaces=['</li>'], imob_excec='BenvegnuEndress', list_regex_excec=['<div id="similares".*', '<div class="col-lg-4 col-md-5 stickyForm.*class="fimRolagemContato">', '<div class="col-lg-4 col-md-5.*class="fimRolagemContato">'])
      sentence = removeTags(page, list_tags_more_spaces=['</li>'], list_remove_tags = [({'Class':'', 'id':'sigaGaleria', 'TAG':''})], imob_excec='Aluga', list_regex_excec=['<div id="footer.*', '<div class="imovel__detalhe__simulador".*', '<div id="similares.*', '<section class="imovel__similar produto".*', '<div class="modal fade modalCookies.*','id="modal-cookies.*','<div class="avisoCookiesFixo.*'])
      page = sentence.lower()
      list_imgs = getImages(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','corretor', 'logo', 'layout'])
      try:
        address_soup = soup.find(class_='sp-address-property').text.strip()
      except:pass
    elif imob == 'Casa Moove Imobiliária' or imob == 'Casa Moove' or imob == 'Ellen Negócios Imobiliários' or imob == 'Formighieri' or imob == 'Costa Lemes Imobiliária' or imob == 'Xarao':
      sentence = removeTags(page, list_tags_more_spaces=['</li>'], list_remove_tags = [({'Class':'', 'id':'sigaGaleria', 'TAG':''})],  imob_excec='CasaMoove', list_regex_excec=['<div id="footer.*', '<div id="similares".*', '<div class="col-lg-4 col-md-5 stickyForm.*class="fimRolagemContato">', '<div class="col-lg-4 col-md-5.*class="fimRolagemContato">', '<div class="modal fade modalCookies.*','id="modal-cookies.*','<div class="avisoCookiesFixo.*'])
      page = sentence.lower()
      list_imgs = getImages(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','corretor', 'logo', 'layout'])
      try:
        address_soup = soup.find(class_='sp-address-property').text.strip()
      except:pass
    elif imob == 'Aporte Imóveis':
      sentence = removeTags(page, list_tags_more_spaces=['</li>'], list_remove_tags = [({'Class':'', 'id':'sigaGaleria', 'TAG':''})], imob_excec='Aporte', list_regex_excec=['<div id="similares".*', '<div class="col-lg-4 col-md-5 stickyForm.*class="fimRolagemContato">', '<div class="col-lg-4 col-md-5.*class="fimRolagemContato">', '<div class="modal fade modalCookies.*','id="modal-cookies.*','<div class="avisoCookiesFixo.*'])
      page = sentence.lower()
      list_imgs = getImagesAriotti(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','corretor', 'logo', 'layout'])
      try:
        address_soup = soup.find(class_='sp-address-property').text.strip()
      except:pass
    elif imob == 'Imobiliária Ariotti':
      sentence = removeTags(page, list_tags_more_spaces=['</li>'], list_remove_tags = [({'Class':'', 'id':'pg_galeria', 'TAG':''})], imob_excec='ariotti', list_regex_excec=['<div id="similares".*', '<div class="col-lg-4 col-md-5 stickyForm.*</button>', '<div class="modal fade modalCookies.*','id="modal-cookies.*','<div class="avisoCookiesFixo.*'])
      page = sentence.lower()
      list_imgs = getImagesAriotti(soup, tag_img='div', attrs_check=['data-src', 'data-exthumbimage'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','corretor', 'logo', 'layout', 'uploads', 'g1'])
      try:
        address_soup = soup.find(class_='sp-address-property').text.strip()
      except:pass
    elif imob == 'Arnel':
      sentence = removeTags(page, list_tags_more_spaces=['</small>'], imob_excec='Arnel', list_regex_excec=['<h2>Semelhantes.*', '<section class="pgl-featured pgl-properties card-semelhantes.*"', 
                                                                                                            '<div class="formEnvia.*', 'card-semelhantes.*' ])
      page = sentence.lower()
      list_imgs = getImages(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','corretor', 'logo', 'layout'])
      try:
        address_soup = soup.find('address').text
      except:pass
    elif imob == 'Verdi' or imob == 'Predial':
      sentence = removeTags(page, list_tags_more_spaces=['</small>'], imob_excec='Verdi', list_regex_excec=['<section id="imovel-form-fixed.*', '<section id="section-indicados".*', '<div class="col-lg-4 col-md-5 stickyForm.*class="fimRolagemContato">', '<div class="col-lg-4 col-md-5.*class="fimRolagemContato">'])
      page = sentence.lower()
      list_imgs = getImages(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','corretor', 'logo', 'layout'])
      try:
        address_soup = soup.find(class_='endereco').text
      except:pass
    elif imob == 'Coratto':
      sentence = removeTags(page, list_tags_more_spaces=['</tbody>'], imob_excec='Coratto', list_regex_excec=['<section id="imoveis-semelhantes".*', '<p class="titulo"></p>.*</aside>'])
      page = sentence.lower()
      list_imgs = getImages(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','corretor', 'logo', 'layout'])

    elif imob.strip() == 'Clube House Petrópolis':
      sentence = removeTags(page, list_tags_more_spaces=['<div class="item-info">'], imob_excec='Clube House', list_regex_excec=['<div class="listing-related text-xs-center".*', '<div class="reference hidden-lg-up">.*'])
      page = sentence.lower()
      list_imgs = getImages(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','corretor', 'logo', 'layout'])
      #VERIFICAR O ENDEREÇO DEPOIS, POIS ESTÁ JUNTO COM O TÍTULO E NÃO TEM UM PADRÃO PARA SEPARAR O ENDEREÇO DO TÍTULO
      try:
        address_soup = soup.find(class_='header-title hidden-lg-up').text
      except:pass
    #elif imob == 'Predial':
    #  sentence = removeTags(page, list_tags_more_spaces=['</small>'], imob_excec='Predial', list_regex_excec=['<section id="imovel-form-fixed.*', '<ul class="list-unstyled main-menu.*</ul>', '<a class="btn-favorito".*</a>',
    #                                                               '<div class="navbar navbar-inverse.*<script>', #'<p class="iterative-notify-random-imovelpage.*</p>',
    #                                                                '<footer.*','<h2 style="line-height: 45px;".*', '<div id="disponivel-credpago.*">'])
    #  page = sentence.lower()
    #  list_imgs = getImages(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','corretor', 'logo', 'layout'])
    #  try:
    #    address_soup = soup.find(class_='endereco').text
    #  except:pass
    elif imob == 'R. Classic':
      sentence = removeTags(page, list_tags_more_spaces=['<div class="item-info">'], imob_excec='R.Classic', list_regex_excec=['<div id="similares".*', '<div class="class="card-broker border-color-contrast".*</div></div>',
                                                                          '<div class="cookies-component-lgpd.*</div>', '<footer.*'])
      page = sentence.lower()
      page = page.replace(' Veja mais  Veja mais                                   Qual o melhor dia e horário para você?                   Invalid date      Invalid date    Invalid date         O local será confirmado pelo corretor               Tenho interesse neste imóvel        Ao enviar concordo com os  termos de uso  e  política de privacidade , para contatar os próximos anunciantes e afirmo ter mais de 18 anos                   Tenho interesse neste imóvel        Ao enviar concordo com os  termos de uso  e  política de privacidade , para contatar os próximos anunciantes e afirmo ter mais de 18 anos','')
      list_imgs = getImages(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','corretor', 'logo', 'layout'])
      try:
        address_soup = soup.find(class_='header-title hidden-lg-up').find(class_='sub').text.strip()
        if address_soup == '' or type(address_soup) == float:
          float(gerar_error_next_except)
      except:
        try:
          address_soup = soup.find(class_='header-title hidden-lg-up').text.strip()
        except:pass
    elif imob == 'Fabiane Imóveis':
      sentence = removeTags(page, list_tags_more_spaces=['</tr>'], imob_excec='Fabianeimoveis', list_regex_excec=['<h1 class="header-text__title">Imóveis semelhantes.*', '<form.*</form>', '<footer.*','<ul class="l-nav__container js-nav js-menu--close.*</ul>','<div class="header-img-box js-mainBanner.*</div>'])
      page = sentence.lower()
      list_imgs = getImages(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','corretor', 'logo', 'layout'])
      try:
        table = soup.find_all('table', class_='table-detail')
        for tr in table:
          trs = tr.find_all('tr')
          for tr in trs:
            if tr.find('td', class_='color-text').text.strip().lower() == 'bairro':
              address_soup = tr.find('td', class_='color-title').text.strip()
      except:pass
    elif imob == 'Grupo Imob':
      sentence = removeTags(page, list_tags_more_spaces=['<div class="item-info">'], imob_excec='GrupoImob', list_regex_excec=['<div class="listing-related".*', '<header.*</header>', '<div class="class="card-broker border-color-contrast".*</div></div>',
                                                                          '<div class="cookies-component-lgpd.*</div>', '<div class="col-md-7 hidden-sm-down box-phones-menu.*</div>','<footer.*'])
      page = sentence.lower()
      page = page.replace(' Veja mais  Veja mais                                   Qual o melhor dia e horário para você?                   Invalid date      Invalid date    Invalid date         O local será confirmado pelo corretor               Tenho interesse neste imóvel        Ao enviar concordo com os  termos de uso  e  política de privacidade , para contatar os próximos anunciantes e afirmo ter mais de 18 anos                   Tenho interesse neste imóvel        Ao enviar concordo com os  termos de uso  e  política de privacidade , para contatar os próximos anunciantes e afirmo ter mais de 18 anos','')
      list_imgs = getImages(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','corretor', 'logo', 'layout'])
      try:
        address_soup = soup.find(class_='header-title hidden-lg-up').find(class_='sub').text.strip()
        if address_soup == '' or type(address_soup) == float:
          float(gerar_error_next_except)
      except:
        try:
          address_soup = soup.find(class_='header-title hidden-lg-up').text.strip()
        except:pass
    elif imob == 'Casa Nostra':
      sentence = removeTags(page, list_tags_more_spaces=['<span class="feature">'], imob_excec='CasaNostra', list_regex_excec=['<footer.*', '<aside id="sidebar.*</aside>', '<form.*</form>'])
      page = sentence.lower()
      list_imgs = getImagesAriotti(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','corretor', 'logo', 'layout'])
      try:
        address_soup = soup.find('p', class_='address').text.strip()
      except:pass
    elif imob == 'Castilhos Nazari' or imob == 'Prima Casa': ###########VERIFICAR PRIMA CASA
      #page = removeTags(page, imob_excec='BenvegnuEndress', list_regex_excec=['<form.*</form>','<ul class="impostos.*','<footer.*', '<ul class="menu-main-menu-1.*</ul>','<section class="rh_property__similar_properties.*</section>','<section class="rh_property__comments.*</section>', '<aside class="rh_sidebar.*</aside>'])
      #page = removeTags(page, list_tags_more_spaces=['<span class="rh_meta_titles">'],  imob_excec='PrimaCasa', list_regex_excec=['<div class="imovel__detalhe__simulador".*', '<section class="imovel__similar produto".*'])
      sentence = removeTags(page, list_tags_more_spaces=['<span class="rh_meta_titles">'],  imob_excec='PrimaCasa', list_regex_excec=['<div class="imovel__detalhe__simulador".*', '<section class="rh_property__similar_properties.*', '<section class="imovel__similar produto".*'])
      sentence = sentence.replace('Comprar     Alugar     Quem Somos      Nossa história     Serviços         Fale Conosco                (54) 99936-4363             atendimento@castilhosnazari.com.br','')
      sentence = sentence.replace('Tipo de Imóvel              Localização               Preço Máximo              Qualquer              Preço Máximo              Código do Imóvel          Quartos              Banheiros              Vagas de Garagem','')
      page = sentence.lower()
      list_imgs = getImages(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','corretor', 'logo', 'layout'])

    elif imob == 'Remax':
      #page = removeTags(page, imob_excec='Remax', list_regex_excec=['<footer.*', '<form.*</form>','<div class="single-contato sidebar.*</div>', '<div class="agente.*</small>'])
      precos = re.findall("\d+, 0, ',', '\.', 'R\$'", page_bruto)
      sentence = removeTags(page, list_tags_more_spaces=['<div class="box-inf-det">', '</li>'], imob_excec='Remax', list_regex_excec=['<footer.*', '<form.*</form>','<div class="single-contato sidebar.*</div>', '<div class="agente.*</small>'])
      sentence += '    '
      for preco in precos:
        sentence += 'R$ '+preco
        sentence += '    '
      sentence = sentence.replace(' Menu                                   Buscar imóveis     Venda seu imóvel      Agências     Seja corretor RE/MAX     Seja Parceiro     Seja um franqueado     Notícias                            Fechar        Buscar imóveis        Venda seu imóvel        Agências        Seja corretor RE/MAX        Seja Parceiro        Seja um franqueado        Notícias','')
      page = sentence.lower()
      list_imgs = getImagesAriotti(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','corretor', 'logo', 'layout'])
      try:
        address_soup = soup.find(class_='container-ficha').find(class_='box-inf-det').text.strip()
      except:pass
    elif imob == 'BP Negócios imobiliários':
      try:
        page = str(soup.find(id='imovel-exibir').find_all(class_='container', recursive=False)[1]).replace('\n',' ').replace('\t',' ')
        page = cleaner.clean_html(page)
      except:
        print("ERROR PAGE", url)
        continue
      sentence = removeTags(page, imob_excec='BP', list_regex_excec=['<footer.*', '<form.*</form>', '<div id="imovel_contato.*</form>'])
      page = sentence.lower()
      list_imgs = getImagesAriotti(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','corretor', 'logo', 'layout', 'no-image'])
      ps = soup.find(class_='imovel_endereco').find_all('p')
      for p in ps:
        if address_soup == 'NENHUM':address_soup = p.text.strip()
        else:address_soup +='&-&'+ p.text.strip()
    elif imob == 'Martins Imóveis':
      try:
        page = str(soup.find(id='imovel-exibir').find_all(class_='container', recursive=False)[0]).replace('\n',' ').replace('\t',' ')
        page = cleaner.clean_html(page)
      except:
        print("ERROR PAGE", url)
        continue
      sentence = removeTags(page, imob_excec='Martins', list_regex_excec=['<footer.*', '<form.*</form>', '<div id="imovel_contato.*</form>'])
      page = sentence.lower()
      list_imgs = getImagesAriotti(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','corretor', 'logo', 'layout', 'no-image'])
      ps = soup.find(class_='imovel_endereco').find_all('p')
      for p in ps:
        if address_soup == 'NENHUM':address_soup = p.text.strip()
        else:address_soup +='&-&'+ p.text.strip()
    elif imob == 'Rosana Lorenzoni Corretora de imóveis':
      try:
        page = str(soup.find(id='imovel-exibir').find_all(class_='container', recursive=False)[0]).replace('\n',' ').replace('\t',' ')
        page = cleaner.clean_html(page)
      except:
        print("ERROR PAGE", url)
        continue
      sentence = removeTags(page, imob_excec='Rosana', list_regex_excec=['<footer.*', '<form.*</form>', '<div id="imovel_contato.*</div>', '<div id="social-media.*</h4>'])
      page = sentence.lower()
      list_imgs = getImagesAriotti(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','corretor', 'logo', 'layout', 'no-image'])
      ps = []
      try:
        ps = soup.find(class_='imovel_endereco').find_all('p')
      except:
        for script in range(len(soup.find_all('script'))):
          try:
            ps = BeautifulSoup(soup.find_all('script')[script].text).find(class_='imovel_endereco').find_all('p')
            break
          except:continue
      for p in ps:
        if address_soup == 'NENHUM':address_soup = p.text.strip()
        else:address_soup +='&-&'+ p.text.strip()

    elif imob == 'Tedesco Imóveis':
      try:
        page = str(soup.find(id='building-display')).replace('\n',' ').replace('\t',' ')
        page = cleaner.clean_html(page)
      except:
        print("ERROR PAGE", url)
        continue
      sentence = removeTags(page, imob_excec='Tedesco', list_regex_excec=['<footer.*','<div id="f-aux.*', '<div class="clearfix" id="building-contact".*</div>', '<form.*</form>', '<div id="imovel_contato.*</div>', '<div id="social-media.*</h4>'])
      page = sentence.lower()
      list_imgs = getImages(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','corretor', 'logo', 'layout','no-image'])
      try:
        address_soup = soup.find('div', class_='layout-row layout-align-center-center').find('h1').find_next_sibling('div').text.replace('\n',' ').replace('\t','').strip()
      except:pass
    elif imob == 'Corretora de Imóveis Camargo':
      try:
        page = str(soup.find(id='building-display')).replace('\n',' ').replace('\t',' ')
        page = cleaner.clean_html(page)
      except:#coratto
        print("ERROR PAGE", url)
        continue
      sentence = removeTags(page, imob_excec='Camargo', list_regex_excec=['<footer.*', '<div class="col-xs-12" id="building-contact">.*', '<form.*</form>', '<div id="imovel_contato.*</div>'])
      page = sentence.lower()
      list_imgs = getImages(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','logo', 'layout','no-image','logo_rodape'])
      try:
        ps = soup.find(class_='col-xs-6 col-md-8').find_all('p')
        address_soup = ps[0].text.replace('\t','').strip().replace('\n','')
      except:pass
    elif imob == 'Carlotto Imóveis':
      try:
        page = str(soup.find(id='building-display')).replace('\n',' ').replace('\t',' ')
        page = cleaner.clean_html(page)
      except:
        print("ERROR PAGE", url)
        continue
      sentence = removeTags(page, imob_excec='IMOB', list_regex_excec=['<footer.*', '<div class="col-xs-12" id="building-contact">.*', '<form.*</form>', '<div id="imovel_contato.*</div>'])
      sentence = sentence.replace('Contato   Entre em contato conosco, preenchendo o formulário abaixo.          Mensagem enviada com sucesso. Agradecemos o Contato.       Ocorreu um erro ao enviar a sua mensagem. Por favor tente novamente.       Nome        Telefone        E-mail        Mensagem        Quero ser contatado por       E-mail      Telefone      Whatsapp       Qualquer uma das opções           Ao clicar em enviar, eu concordo em receber comunicações da Imobiliária, utilizando os dados que forneci neste formulário','')
      page = sentence.lower()
      list_imgs = getImages(soup, tag_img='img', attrs_check=['data-src', 'src'], urls_checks=['.jpg', '.jpeg'], check_excessao=['usua','logo', 'layout','no-image','logo_rodape'])
      ps = soup.find(class_='grid-body').find_all('p')
      for p in ps:
        try:
          if 'endereco' in unidecode(p.find('strong').text.strip().lower()):
            if address_soup == 'NENHUM':address_soup = p.text.replace(p.find('strong').text, '').replace('()','').strip().replace('\n',' ')
            else:address_soup +='&-&'+p.text.replace(p.find('strong').text, '').replace('()','').strip().replace('\n',' ')
        except:pass
        try:
          if 'bairro' in unidecode(p.find('strong').text.strip().lower()):
            if address_soup == 'NENHUM':address_soup = p.text.strip()
            else:address_soup +='&-&'+p.text.strip()
            break
        except:pass
    else:
      print("IMOB NÂO TEM")
      continue
  
    """
      FUNÇÕES DO NER - PRIMEIRAS TENTATIVAS COM REGEX E O NER 4...7
    """
    if sentence == 'NENHUM':
      print("OPAAAA SENTENCE")
      break
    
    suites_regex, dorms_regex, vagas_regex, banheiros_regex, dorms_spacy_rg,suites_spacy_rg,banheiro_spacy_rg,vagas_spacy_rg = regexSpacy1(page)
    list_coordenadas_regex, address_regex_iframe_return = coordenadasRegex(soup_regex = soup, imob = imob)
    suites1, suites2, dorms, vagas, quartos, banheiros_r = findWordPerIndice(page)

    desc_ner = sentence.lower()
    desc_ner45 = sentence

    area_m = re.findall('\d+[,.]?\d+\s?m²', desc_ner)
    area_ha = re.findall('\d+[,.]?\d+\s?ha', desc_ner)
    if area_ha == []:area_ha = re.findall('\d+[,.]?\d+\s?hect', desc_ner)

    desc_ner = desc_ner.replace('_', ' ')
    if findWordsAndIndices(desc_ner, 'sui', 'SUITES', sub_word_padrao='suítes', option_ner = 2, sentence_ner=desc_ner) != (0, 0): desc_ner = findWordsAndIndices(desc_ner, 'sui', 'SUITES', sub_word_padrao='suítes', option_ner = 2, sentence_ner=desc_ner)
    if findWordsAndIndices(desc_ner, 'suí', 'SUITES', sub_word_padrao='suítes', option_ner = 2, sentence_ner=desc_ner) != (0, 0): desc_ner = findWordsAndIndices(desc_ner, 'sui', 'SUITES', sub_word_padrao='suítes', option_ner = 2, sentence_ner=desc_ner)
    if findWordsAndIndices(desc_ner, 'vaga', 'VAGAS', sub_word_padrao='vagas', option_ner = 2, sentence_ner=desc_ner) != (0, 0): desc_ner = findWordsAndIndices(desc_ner, 'vaga', 'VAGAS', sub_word_padrao='vagas', option_ner = 2, sentence_ner=desc_ner)
    if findWordsAndIndices(desc_ner, 'dorm', 'DORMS', sub_word_padrao='dormitorios', option_ner = 2, sentence_ner=desc_ner) != (0, 0): desc_ner = findWordsAndIndices(desc_ner, 'dorm', 'DORMS', sub_word_padrao='dormitorios', option_ner = 2, sentence_ner=desc_ner)
    if findWordsAndIndices(desc_ner, 'banheiro', 'BANHEIROS', sub_word_padrao='banheiros', option_ner = 2, sentence_ner=desc_ner) != (0,0): desc_ner = findWordsAndIndices(desc_ner, 'banheiro', 'BANHEIROS', sub_word_padrao='banheiros', option_ner = 2, sentence_ner=desc_ner)

    #dados estão apenas proporcionais ------------------------------------------------------------------------
    #print("\n-------SPACY NER 4-------")
    cod4, tipo4, price4, dorms4, suites4, area4, vagas4, banheiros4, address4, sentences = [],[],[],[],[],[],[],[],[],[]
    for model_type in [desc_ner45, desc_ner45.lower(), re.sub('\s+',' ',desc_ner45), re.sub('\s+',' ',desc_ner), re.sub('\s+',' ',desc_ner.lower()), desc_ner.lower(), tokenize_sentence(page.lower()), tokenize_sentence(desc_ner.lower())]:
        doc4 = nlp4(model_type)
        sentences.append(doc4)
        for ent in doc4.ents:
          if ent.label_ == 'TIPO':tipo4.append(ent.text)
          if ent.label_ == 'COD':cod4.append(ent.text)
          if ent.label_ == 'PRICE':price4.append(ent.text)
          if ent.label_ == 'DORMS':dorms4.append(ent.text)
          if ent.label_ == 'SUITES':suites4.append(ent.text)
          if ent.label_ == 'AREA':area4.append(ent.text)
          if ent.label_ == 'VAGAS':vagas4.append(ent.text)
          if ent.label_ == 'BANHEIROS':vagas4.append(ent.text)
          if ent.label_ == 'ADDRESS':address4.append(ent.text)
          #print(ent.text, ent.label_)

    #dados proporcionais e ajustados (dorm = dormitorios) --------------------------------------------------------------------------
      #print("\n-------SPACY NER 5-------")
    cod5, tipo5, price5, dorms5, suites5, area5, vagas5, banheiros5, address5 = [],[],[],[],[],[],[],[],[]
    for model_type in [desc_ner45, desc_ner45.lower(), re.sub('\s+',' ',desc_ner45), re.sub('\s+',' ',desc_ner), re.sub('\s+',' ',desc_ner.lower()), desc_ner.lower(), tokenize_sentence(page.lower()), tokenize_sentence(desc_ner.lower())]:
        doc5 = nlp5(model_type)
        #doc5 = nlp5(re.sub('\s+',' ',desc_ner45)) #padrão testado antes
        for ent in doc5.ents:
          if ent.label_ == 'TIPO':tipo5.append(ent.text)
          if ent.label_ == 'COD':cod5.append(ent.text)
          if ent.label_ == 'PRICE':price5.append(ent.text)
          if ent.label_ == 'DORMS':dorms5.append(ent.text)
          if ent.label_ == 'SUITES':suites5.append(ent.text)
          if ent.label_ == 'AREA':area5.append(ent.text)
          if ent.label_ == 'VAGAS':vagas5.append(ent.text)
          if ent.label_ == 'BANHEIROS':vagas5.append(ent.text)
          if ent.label_ == 'ADDRESS':address5.append(ent.text)


    #remove stop words e padroniza os dados --------------------------------------------------------------------------------------
    desc_ner = page.replace('_', ' ')
    if findWordsAndIndices(desc_ner, 'sui', 'SUITES', sub_word_padrao='suítes', option_ner = 2, sentence_ner=desc_ner) != (0,0): desc_ner = findWordsAndIndices(desc_ner, 'sui', 'SUITES', sub_word_padrao='suítes', option_ner = 2, sentence_ner=desc_ner)
    if findWordsAndIndices(desc_ner, 'suí', 'SUITES', sub_word_padrao='suítes', option_ner = 2, sentence_ner=desc_ner) != (0,0): desc_ner = findWordsAndIndices(desc_ner, 'sui', 'SUITES', sub_word_padrao='suítes', option_ner = 2, sentence_ner=desc_ner)
    if findWordsAndIndices(desc_ner, 'vaga', 'VAGAS', sub_word_padrao='vagas', option_ner = 2, sentence_ner=desc_ner) != (0,0): desc_ner = findWordsAndIndices(desc_ner, 'vaga', 'VAGAS', sub_word_padrao='vagas', option_ner = 2, sentence_ner=desc_ner)
    if findWordsAndIndices(desc_ner, 'dorm', 'DORMS', sub_word_padrao='dormitorios', option_ner = 2, sentence_ner=desc_ner) != (0,0): desc_ner = findWordsAndIndices(desc_ner, 'dorm', 'DORMS', sub_word_padrao='dormitorios', option_ner = 2, sentence_ner=desc_ner)
    if findWordsAndIndices(desc_ner, 'banheiro', 'BANHEIRO', sub_word_padrao='banheiro', option_ner = 2, sentence_ner=desc_ner) != (0,0): desc_ner = findWordsAndIndices(desc_ner, 'banheiro', 'BANHEIRO', sub_word_padrao='banheiro', option_ner = 2, sentence_ner=desc_ner)

    cod6, tipo6, price6, dorms6, suites6, area6, vagas6, banheiros6, address6, =  [],[],[],[],[],[],[],[],[]
    for model_type in [desc_ner45, desc_ner45.lower(), re.sub('\s+',' ',desc_ner45), re.sub('\s+',' ',desc_ner), re.sub('\s+',' ',desc_ner.lower()), desc_ner.lower(), tokenize_sentence(page.lower()), tokenize_sentence(desc_ner.lower())]:
        doc6 = nlp6(model_type)
        for ent in doc6.ents:
          if ent.label_ == 'TIPO':tipo6.append(ent.text)
          if ent.label_ == 'COD':cod6.append(ent.text)
          if ent.label_ == 'PRICE':price6.append(ent.text)
          if ent.label_ == 'DORMS':dorms6.append(ent.text)
          if ent.label_ == 'SUITES':suites6.append(ent.text)
          if ent.label_ == 'AREA':area6.append(ent.text)
          if ent.label_ == 'VAGAS':vagas6.append(ent.text)
          if ent.label_ == 'BANHEIROS':vagas6.append(ent.text)
          if ent.label_ == 'ADDRESS':address6.append(ent.text)

    cod7, tipo7, price7, dorms7, suites7, area7, vagas7, banheiros7, address7 = [],[],[],[],[],[],[],[],[]
    for model_type in [page, desc_ner45, desc_ner45.lower(), re.sub('\s+',' ',desc_ner45), re.sub('\s+',' ',desc_ner), re.sub('\s+',' ',desc_ner.lower()), desc_ner.lower(), tokenize_sentence(page.lower()), tokenize_sentence(desc_ner.lower())]:
      doc7 = nlp7(model_type) 
      for ent in doc7.ents:
        if ent.label_ == 'TIPO':tipo7.append(ent.text)
        if ent.label_ == 'COD':cod7.append(ent.text)
        if ent.label_ == 'PRICE':price7.append(ent.text)
        if ent.label_ == 'DORMS':dorms7.append(ent.text)
        if ent.label_ == 'SUITES':suites7.append(ent.text)
        if ent.label_ == 'AREA':area7.append(ent.text)
        if ent.label_ == 'VAGAS':vagas7.append(ent.text)
        if ent.label_ == 'BANHEIROS':vagas7.append(ent.text)
        if ent.label_ == 'ADDRESS':address7.append(ent.text)

    #print('\n NER 7.1----------------')
    cod71, tipo71, price71, dorms71, suites71, area71, vagas71, banheiros71, address71 = [],[],[],[],[],[],[],[],[]
    #for ent in doc.ents:
    for model_type in [page, desc_ner45, desc_ner45.lower(), re.sub('\s+',' ',desc_ner45), re.sub('\s+',' ',desc_ner), re.sub('\s+',' ',desc_ner.lower()), desc_ner.lower(), tokenize_sentence(page.lower()), tokenize_sentence(desc_ner.lower())]:
      doc71 = nlp71(model_type)
      for ent in doc71.ents:
        #print(ent.text, ent.label_)
        if ent.label_ == 'TIPO':tipo71.append(ent.text)
        if ent.label_ == 'COD':cod71.append(ent.text)
        if ent.label_ == 'PRICE':price71.append(ent.text)
        if ent.label_ == 'DORMS':dorms71.append(ent.text)
        if ent.label_ == 'SUITES':suites71.append(ent.text)
        if ent.label_ == 'AREA':area71.append(ent.text)
        if ent.label_ == 'VAGAS':vagas71.append(ent.text)
        if ent.label_ == 'BANHEIROS':vagas71.append(ent.text)
        if ent.label_ == 'ADDRESS':address71.append(ent.text)

    list_bairros_regex = getBairros(page)

    list_ner.append([{'URL':url, 'Imobiliária': imob, 'Sentence': sentence, 'Imagens': list_imgs}, {'Regex': {'Suites':suites2, 'Banheiros': banheiros_r, 'COORDS': list_coordenadas_regex, 'ADDRESS': address_regex_iframe_return,
                                                'Dorms':dorms, 'Vagas':vagas, 'Quartos':quartos, 'AREA m2':area_m, 'AREA ha': area_ha}},
                                      {'REGEX spacy':{'Suítes':suites_regex,  'Bairro':list_bairros_regex,
                                          'Dorms':dorms_regex, 'Vagas':vagas_regex, 'Banheiros':banheiros_regex}},
                                      {'REGEX spacy 2':{'Suítes':suites_spacy_rg,
                                                  'Dorms':dorms_spacy_rg, 'Vagas':vagas_spacy_rg, 'Banheiros':banheiro_spacy_rg}},
                                      {'NER 4':
                                            {'Suites':suites4, 'COD':cod4,
                                            'Tipo':tipo4,'Preco':price4, 'Dorms':dorms4, 'Vagas':vagas4, 'AREA':area4, 'BANHEIRO':banheiros4, 'ADDRESS':address4}},
                                      {'NER 5':
                                            {'Suites':suites5, 'COD':cod5,
                                            'Tipo':tipo5,'Preco':price5, 'Dorms':dorms5, 'Vagas':vagas5, 'AREA':area5, 'BANHEIRO':banheiros5, 'ADDRESS':address5}},
                                      {'NER 6':
                                            {'Suites':suites6, 'COD':cod6,
                                            'Tipo':tipo6,'Preco':price6, 'Dorms':dorms6, 'Vagas':vagas6, 'AREA':area6, 'BANHEIRO':banheiros6, 'ADDRESS':address6}},
                                      {'NER 7':
                                            {'Suites':suites7, 'COD':cod7,
                                            'Tipo':tipo7,'Preco':price7, 'Dorms':dorms7, 'Vagas':vagas7, 'AREA':area7, 'BANHEIRO':banheiros7, 'ADDRESS':address7}},
                                      {'NER 7.1':
                                            {'Suites':suites71, 'COD':cod71,
                                            'Tipo':tipo71,'Preco':price71, 'Dorms':dorms71, 'Vagas':vagas71, 'AREA':area71, 'BANHEIRO':banheiros71, 'ADDRESS':address71}},
                                      #{'Humano Verificar':human_verify}
                    ])



    """
      FUNÇÕES NOVAS DO REGEX E CARACTER POR CARACTER
    """
    #print(indice_df, sentence)
    sentence_not_space =  unidecode(sentence.lower().replace('(','').replace(')','').replace(':','').replace('\xa0',''))
    sentence_mod = re.sub(',', ' , ', re.sub('\s+', ' ', unidecode(sentence.lower().replace('(','').replace(')','').replace(':','').replace('\xa0',''))))


    dorms, quartos, suites, vagas, banheiros, garagem, carros = 'NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM',
    area, area_priv, area_terreno, area_contruida, area_total, area_global, area_util = 'NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM',

    dorms, quartos, suites, vagas, banheiros, garagem, carros = tagsShow(sentence, tipo = 'comodos')
    area, area_priv, area_terreno, area_contruida, area_total, area_global, area_util = tagsShow(sentence, tipo = 'areas')
    

    dorms_rgx_only, suites_rgx_only, vagas_rgx_only, banheiro_rgx_only, quarto_rgx_only, garagem_rgx_only = 'NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM'
    dorms_rgx_only2, suites_rgx_only2, vagas_rgx_only2, banheiro_rgx_only2, quartos_rgx_only2, garagem_rgx_only2, carros_rgx_only2 = 'NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM'

    """alternativa regex que pega qualquer expressão que tenha número e palavras juntos: 2 dorms 3 suítes 4 banheiros"""
    _, _, dorms_rgx_only, suites_rgx_only, vagas_rgx_only, banheiro_rgx_only, quarto_rgx_only, garagem_rgx_only = returnComodosJuntos(sentence)

    """quando há vários cômodos juntos, que é especificado (precisa saber qual cômodo quer, como dorms, suítes, banheiros ...), e não se sabe qual número é de qual cômodo. 
      comodosChavesJuntos tenta solucionar isso. Em muitos casos é efetivo. Sua melhoria é o comodosJuntosNEW"""
    dorms_rgx_only2, suites_rgx_only2, vagas_rgx_only2, banheiro_rgx_only2, quartos_rgx_only2, garagem_rgx_only2, carros_rgx_only2 = comodosChavesJuntos(sentence)

    """quando há na sentença o cômodo que procurado e ele está sozinho com o número, seja do lado esquerdo, seja do lado direito. Por exemplo, 2 dorms ou dorms 2, não havendo outro número
      que se possa juntar-se ao cômodo pesquisado"""
    dorms_unique, quartos_unique, suites_unique, vagas_unique, banheiros_unique, garagem_unique, carros_unique, box_unique = comodosUnique(sentence)

    """ a partir da extração, é colocado um espaço em cada tag que tiver. Logo, em muitas vezes, nessas extrações, quanto maior o número de espaços, quer dizer que teve mais 
      tags, logo, havia uma separação entre dorms e suítes, por exemplo. Isso significa que quanto menor o número de espaços que tinha entre um cômodo específico e um número, 
      em muitos casos, seria o valor correto para aquele cômodo. Entretanto, houve alguns casos em que isso não ocorre, como na imobiliária coratto. Dessa forma, é preciso na extração
      melhorar essa parte, colocar mais espaços nas tags específicas que separam um cômodo do outro
    """
    dorm_rgx_tag_space, suite_rgx_tag_space, vaga_rgx_tag_space, banheiro_rgx_tag_space, quarto_rgx_tag_space, garagem_rgx_tag_space, carro_rgx_tag_space = 'NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM'   
    dorm_rgx_tag_space, suite_rgx_tag_space, vaga_rgx_tag_space, banheiro_rgx_tag_space, quarto_rgx_tag_space, garagem_rgx_tag_space, carro_rgx_tag_space = returnTagsContSpace(sentence_not_space)

    """ quando tem cômodos juntos e não sabe qual número é de qual cômodo. Dessa forma, o comodosJuntosNEW tenta solucionar esse problema - mais novo"""
    dorms_rgx_only3, suites_rgx_only3, vagas_rgx_only3, banheiro_rgx_only3, quartos_rgx_only3, garagem_rgx_only3, carros_rgx_only3 = comodosJuntosNEW(sentence)

    #'only carros REGEX':carros_rgx_only,

    dorms_or, quartos_or, suites_or, banheiros_or, vagas_or, garagem_or, carros_or, box_or = regexOU(sentence)


    return_areas = returnArea(url, sentence)
    str_area = 'NENHUM'
    for ar in return_areas:
      if str_area =='NENHUM':str_area = ar  
      else: str_area +='&-&'+ar
    #df_sentences['only area REGEX'].iloc[indice_df] = str_area

    return_areas_priv = returnAreaPrivativa(url, sentence)
    str_area_priv = 'NENHUM'
    for ar in return_areas_priv:
      if str_area_priv =='NENHUM':str_area_priv = ar  
      else: str_area_priv +='&-&'+ar
    #df_sentences['only area priv REGEX'].iloc[indice_df] = str_area_priv

    return_areas_terreno = returnAreaTerreno(url, sentence)
    str_area_priv = 'NENHUM'
    for ar in return_areas_terreno:
      if str_area_priv =='NENHUM':str_area_priv = ar  
      else: str_area_priv +='&-&'+ar
    #df_sentences['only area terreno REGEX'].iloc[indice_df] = str_area_priv

    #NLP8 - PRECO
    str_preco_predict_ner = 'NENHUM'
    doc = nlp8(sentence)
    for ent in doc.ents:
      if ent.label_ == 'PRECO':
        #print(ent.text)
        if str_preco_predict_ner == 'NENHUM':str_preco_predict_ner = ent.text
        else:str_preco_predict_ner +='&-&'+ent.text
    #df_sentences['preço NER'].iloc[indice_df] = str_preco_predict_ner

    #NLP9 - DORMS
    str_dorms_predict_ner = 'NENHUM'
    doc = nlp9(sentence_mod)
    for ent in doc.ents:
      if ent.label_ == 'DORMS':
        if str_dorms_predict_ner == 'NENHUM':str_dorms_predict_ner = ent.text
        else:str_dorms_predict_ner +='&-&'+ent.text
    #df_sentences['dorms NER'].iloc[indice_df] = str_dorms_predict_ner

    #NLP9 - QUARTOS
    str_quartos_predict_ner = 'NENHUM'
    doc = nlp9(sentence_mod)
    for ent in doc.ents:
      if ent.label_ == 'QUARTOS':
        if str_quartos_predict_ner == 'NENHUM':str_quartos_predict_ner = ent.text
        else:str_quartos_predict_ner +='&-&'+ent.text
    #df_sentences['quartos NER'].iloc[indice_df] = str_quartos_predict_ner
      
    #NLP10 - VAGAS
    str_vagas_predict_ner = 'NENHUM'
    doc = nlp10(sentence_mod)
    for ent in doc.ents:
      if ent.label_ == 'VAGAS':
        if str_vagas_predict_ner == 'NENHUM':str_vagas_predict_ner = ent.text
        else:str_vagas_predict_ner +='&-&'+ent.text
    #df_sentences['vagas NER'].iloc[indice_df] = str_vagas_predict_ner

    #NLP11 - AREA
    str_area_predict_ner = 'NENHUM'
    doc = nlp11(sentence_mod)
    for ent in doc.ents:
      if ent.label_ == 'AREA':
        if str_area_predict_ner == 'NENHUM':str_area_predict_ner = ent.text
        else:str_area_predict_ner +='&-&'+ent.text
    #df_sentences['area NER'].iloc[indice_df] = str_area_predict_ner

    #NLP12 - SUITES
    str_suites_predict_ner = 'NENHUM'
    doc = nlp12(sentence_mod)
    for ent in doc.ents:
      if ent.label_ == 'SUITES':
        if str_suites_predict_ner == 'NENHUM':str_suites_predict_ner = ent.text
        else:str_suites_predict_ner +='&-&'+ent.text
    #df_sentences['suites NER'].iloc[indice_df] = str_suites_predict_ner

    #NLP12 - BANHEIROS
    str_banheiros_predict_ner = 'NENHUM'
    doc = nlp12(sentence_mod)
    for ent in doc.ents:
      if ent.label_ == 'BANHEIROS':
        if str_banheiros_predict_ner == 'NENHUM':str_banheiros_predict_ner = ent.text
        else:str_banheiros_predict_ner +='&-&'+ent.text
    #df_sentences['banheiros NER'].iloc[indice_df] = str_banheiros_predict_ner

    value_suites_regex, value_correct_suites_regex, value_banheiro_regex, value_correct_banheiro_regex, value_vagas_regex, value_correct_vagas_regex, value_quartos_regex, value_correct_quartos_regex='NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM','NENHUM',
    value_dorms_regex, value_correct_dorms_regex, value_regex_price_reais, value_area_total_regex, value_correct_area_priv_correct_regex = 'NENHUM','NENHUM','NENHUM','NENHUM','NENHUM',

    value_preco_regex, str_preco_regex_end = 'NENHUM', 'NENHUM'
    str_area_regex_area = 'NENHUM'

    for TAG in ['DORMS', 'QUARTOS', 'VAGAS', 'SUITES', 'BANHEIROS', 'AREA', 'PRECO']:
      tag_regex, value ='NULL', []

      if TAG == 'PRECO':
        s = re.sub('mil', '.000,00', sentence_mod)
        value_preco_regex = re.findall('\(?r\$\)?[:\s*]*\d+[.,]?[\d+]*?[.,]?[\d+]*[.,]?[\d+]*[.,]?[\d+]*', s.lower())
        #if value_preco_regex != []:value_preco_regex = value_preco_regex[0]
        if value_preco_regex != []:
          for vp in value_preco_regex:
            if str_preco_regex_end == 'NENHUM':
              str_preco_regex_end = vp
            else: str_preco_regex_end +='&-&'+vp
          #df_sentences['Preço All'].iloc[indice_df] = str_preco_regex_end
        #df_sentences['preco REGEX'].iloc[indice_df] = value_preco_regex
      elif TAG == 'AREA':
        value = re.findall('\S*\d+[,.]?\d+?\s*[m2|m²|m 2|m ²]\S*', sentence_not_space)
        #value_test_correct = value
        #value2 = re.findall('\S*\d+[,.]?\d+m?\s+?m?x\s+?\d+[,.]?\d+m?\s?m?\s', sentence_not_space)
        #value += value2
        value = re.findall('\d+.?,?\d+?\s*?m²', sentence_not_space)
        value12 = re.findall('\d+.?,?\d+?\s*?m2', sentence_not_space)
        if value12 != []:value +=value12
        value13 = re.findall('\d+.?,?\d+?\s*?m ²', sentence_not_space)
        if value13 != []:value +=value13
        value14 = re.findall('\d+.?,?\d+?\s*?m m', sentence_not_space)
        if value14 != []:value +=value14
        for vp in value:
          if str_area_regex_area == 'NENHUM':
            str_area_regex_area = vp
          else: str_area_regex_area +='&-&'+vp
        #df_sentences['area REGEX'].iloc[indice_df] = str_area_regex_area
        continue
      else:
        if TAG == 'DORMS': tag_regex = 'dorm'
        elif TAG == 'QUARTOS': tag_regex = 'quarto'
        elif TAG == 'VAGAS': tag_regex = 'vaga'
        elif TAG == 'SUITES': tag_regex = 'suite'
        elif TAG == 'BANHEIROS': tag_regex = 'banheiro'

        value = re.findall('\S*\d{1,2}'+f'\s*?{tag_regex}\S*', unidecode(sentence_not_space)) #2 dorms
        value2 = re.findall(f'\S*{tag_regex}[\w]*\s+?'+'\d{1,2}\S*', unidecode(sentence_not_space)) #dorms 2

        value += value2
    
      values = 'NENHUM'
      if value != [] and TAG != 'PRECO' and TAG != 'AREA':
        for v in value:
          v = v.strip()
          ns = re.findall('\d+', v)
          if values == 'NENHUM':
            values = ns[0]
          else:values +='&-&'+ns[0]

      area_priv, area_total = 'NENHUM', 'NENHUM'
      value_reais = 'NENHUM'
      value_correct = 'NENHUM'
      if TAG != 'PRECO' and TAG != 'AREA':
        list_vs = []
        for v in value:
          v = v.strip()
          list_vs.append((  len(re.findall('\s', v)), v))
        if len(list_vs) > 0:
          list_vs.sort()
          for vs in list_vs:
            val = re.findall('\d+', vs[1])
            if len(val) > 1:
              value_correct = 'REVISÃO'
              break
            else:
              if ('m2' not in val[0] or 'm²' not in val[0]) and ('r$' not in val[0] and TAG != 'AREA'): value_correct = val[0]
              else: value_correct = 'REVISÃO'
              break

      value_reais = 'NENHUM'
      if TAG == 'PRECO':
          list_reais = []
          for r in value:
            r2 = re.sub('\s+','',r.replace('r$','').strip())
            if len(r2) >= 3:
              if r2[-3] != ',': r2 +=',00'
              r2 = r2.replace(',','').replace('.','')
              list_reais.append((float(r2), r))
            else:
              try:
                r2 +=',00'
                r2 = r2.replace(',','').replace('.','')
                list_reais.append((float(r2), r))
              except:pass
          if len(list_reais) > 0:
            list_reais.sort()
            value_reais = list_reais[-1][1]

      elif TAG == 'AREA':
          list_area = []
          for r in value:
            a2 = re.findall('\d+[,.]?\d+', r)
            if len(re.findall('[.,]', a2[0])) > 1:pass
              #list_area = 'REVISÃO'
            else:
              a2 = a2[0].replace(',','').replace('.','')
              a2 = float(a2)
              list_area.append(a2)        
          if len(list_area) > 0:
            list_area.sort()
          if len(list_area) == 1:
            area_total = list_area[-1]
            area_priv = list_area[-1]
          elif len(list_area) > 1:
            area_total = list_area[-1]
            for ap in list_area:
              if ap > 0 and ap > 22:
                area_priv = ap
                break

      if TAG == 'DORMS': 
        value_dorms_regex = values
        value_correct_dorms_regex = value_correct
      elif TAG == 'PRECO':
        value_regex_price_reais = value_reais
      elif TAG == 'AREA':
        value_area_total_regex = values
        value_correct_area_priv_correct_regex = area_priv
      elif TAG == 'QUARTOS':
        value_quartos_regex = values
        value_correct_quartos_regex = value_correct
      elif TAG == 'VAGAS':
        value_vagas_regex = values
        value_correct_vagas_regex = value_correct
      elif TAG == 'SUITES':
        value_suites_regex = values
        value_correct_suites_regex = value_correct
      elif TAG == 'BANHEIROS':
        value_banheiro_regex = values
        value_correct_banheiro_regex = value_correct

    df_regex_extract = df_regex_extract.append({'Dorms Show':dorms,'Quartos Show': quartos,'Suites Show':suites,'Vagas Show':vagas,'Banheiros Show':banheiros,'Garagem Show':garagem,'Carros Show':carros,
      'Area Show':area,'Area Priv Show':area_priv,'Area Terreno Show':area_terreno,'Area Global Show':area_global,'Area Útil Show':area_util,'Area Construída Show':area_contruida,'Area Total Show':area_total,
      'dorms REGEX sentence':dorm_rgx_tag_space,'only dorms REGEX':dorms_rgx_only,'only dorms REGEX 2':dorms_rgx_only2,'dorms unique REGEX':dorms_unique,'only dorms REGEX 3':dorms_rgx_only3,
      'quartos REGEX sentence':quarto_rgx_tag_space,'only quartos REGEX':quarto_rgx_only,'only quartos REGEX 2':quartos_rgx_only2,'only quartos REGEX 3':quartos_rgx_only3,'quartos unique REGEX':quartos_unique,
      'vagas REGEX sentence':vaga_rgx_tag_space, 'carro REGEX sentence':carro_rgx_tag_space, 'garagem REGEX sentence':garagem_rgx_tag_space,'only vagas REGEX':vagas_rgx_only,'only vagas REGEX 2':vagas_rgx_only2,
      'only vagas REGEX 3':vagas_rgx_only3,'only garagem REGEX':garagem_rgx_only,'only garagem REGEX 2':garagem_rgx_only2,'only garagem REGEX 3':garagem_rgx_only3,'only carros REGEX 2':carros_rgx_only2,
      'only carros REGEX 3':carros_rgx_only3,'vagas unique REGEX':vagas_unique,'garagem unique REGEX':garagem_unique,'carros unique REGEX':carros_unique,
      'suites REGEX sentence':suite_rgx_tag_space,'only suites REGEX':suites_rgx_only,'only suites REGEX 2':suites_rgx_only2,'only suites REGEX 3':suites_rgx_only3,'suites unique REGEX':suites_unique,
      'banheiros REGEX sentence':banheiro_rgx_tag_space,'only banheiros REGEX':banheiro_rgx_only,'only banheiros REGEX 2':banheiro_rgx_only2,'only banheiros REGEX 3':banheiro_rgx_only3,'banheiros unique REGEX':banheiros_unique,
      'Dorms OU': dorms_or,'Quartos OU': quartos_or,'Suites OU': suites_or,'Banheiros OU': banheiros_or,'Vagas OU': vagas_or,'Garagem OU': garagem_or,'Carros OU': carros_or,'Box OU': box_or,
      'only area REGEX':str_area, 'only area priv REGEX':str_area_priv, 'only area terreno REGEX':str_area_priv, 'area REGEX':str_area_regex_area,
      'banheiros NER':str_banheiros_predict_ner, 'suites NER':str_suites_predict_ner, 'area NER':str_area_predict_ner, 'vagas NER':str_vagas_predict_ner, 'quartos NER':str_quartos_predict_ner,
      'dorms NER':str_dorms_predict_ner, 'preço NER': str_preco_predict_ner, 'dorms REGEX': value_dorms_regex, 'dorms REGEX correct':value_correct_dorms_regex,
      'banheiros REGEX':value_banheiro_regex, 'banheiros REGEX correct': value_correct_banheiro_regex, 'suites REGEX': value_suites_regex, 'suites REGEX correct':value_correct_suites_regex,
      'vagas REGEX':value_vagas_regex, 'vagas REGEX correct': value_correct_vagas_regex, 'quartos REGEX': value_quartos_regex, 'quartos REGEX correct':value_correct_quartos_regex,
      'area REGEX correct':value_area_total_regex, 'area priv REGEX correct': value_correct_area_priv_correct_regex, 'preco REGEX correct':value_regex_price_reais, 
      'Preço All': str_preco_regex_end, 'preco REGEX': value_preco_regex,
      'Sentence':sentence, 'Imobiliária':imob,
      'Tipo':tipo, 'Situacao':situacao, 'URL': url, 'Endereco Soup': address_soup, 'Pagina': page_bruto}, ignore_index = True)
    list_urls_novos.append(url)
    cont_imobs_extraction_now += 1

    if cont_imobs_extraction_now % 80 == 0:
      print("save")
      df_regex_extract.to_csv(f'{name_save_file_csv_and_json}.csv', sep='\t')
      with open(f'{name_save_file_csv_and_json}.json', 'w') as f:
        json.dump(list_ner, f)
      #data = df_regex_extract.copy()
      #data = data[np.isin(data['URL'].to_numpy(), list_urls_novos)]
      #data.loc[[x in list_urls_novos for x in data['URL'].tolist()]]
      #data = pd.read_csv('./DataFrames/2-df_final_regex-2202-02.csv', sep = '\t', lineterminator = '\n')
      #client = pymongo.MongoClient("mongodb://localhost:27017/")
      #db = client['coop']
      #collection = db['imoveis3']
      #data.reset_index(inplace=True)
      #data_dict = data.to_dict("records")
      #collection.insert_many(data_dict)

    time.sleep(randint(4,8))
  df_regex_extract.to_csv(f'{name_save_file_csv_and_json}.csv', sep='\t')
  with open(f'{name_save_file_csv_and_json}.json', 'w') as f:
    json.dump(list_ner, f)
  
  #salva em permanente os links que foram extraídos
  timestamp_att_end = checkTimesIsOpenFile(arquivo_urls_config) #checa o tempo antes de abrir e salvar
  urls_config = pd.read_csv(name_urls_config, sep='\t')
  for url_new in list_urls_novos:
    for ic in range(urls_config.shape[0]):
      if url_new == urls_config['URL'].iloc[ic] and (urls_config['Situacao'].iloc[ic] == 'Em extracao' or urls_config['Situacao'].iloc[ic] == 'Revisao'):
        urls_config['Situacao'].iloc[ic] = 'OK'
        break
  urls_config.to_csv(name_urls_config, sep='\t')
df_regex_extract.to_csv(f'{name_save_file_csv_and_json}.csv', sep='\t')
with open(f'{name_save_file_csv_and_json}.json', 'w') as f:
  json.dump(list_ner, f)

TEMPO 60s
(21109, 5) - URLS CONFIG: 21425
Selecionou (0,)
TEMPO DESDE O ÚLTIMO 266.3738708496094
ADICIONOU 0
TEMPO DESDE O ÚLTIMO 0.5146193504333496
loading seconds urls_config


KeyboardInterrupt: ignored

In [ ]:
df_regex_extract.to_csv(f'{name_save_file_csv_and_json}.csv', sep='\t')
with open(f'{name_save_file_csv_and_json}.json', 'w') as f:
  json.dump(list_ner, f)

In [ ]:
len('sdfsd fsda fsaddsfgd5sa 0f456sa 0df5as d5f4sd4f as4d5f 4sd5f048f 04sd80f4d f4sd fs d4f 4d5f as50d4fs5')

101

In [ ]:
df_regex_extract.to_csv('/content/gdrive/My Drive/NER/DataFrames/Extracao/df_extraction_new_Parte-1.csv', sep='\t')
with open('/content/gdrive/My Drive/NER/DataFrames/Extracao/json_extraction_new_Parte-1.json', 'w') as f: #17-03;17-03-2;22-03;25-03
  json.dump(list_ner, f)